import pandas as pd
import numpy as np
from scipy import stats

#############################
# 1. Load and merge results #
#############################

# If your files are Excel:
gemma_df = pd.read_excel("Gemma_Resultss.xlsx")  # <-- adjust sheet name
gpt_df   = pd.read_excel("Results_GPTa.xlsx")    # <-- adjust sheet name

# Or if you exported to CSV:
#gemma_df = pd.read_csv("Gemma_Results.csv")
#gpt_df   = pd.read_csv("Results_GPT.csv")

# Add model labels (if not already present)
gemma_df["model"] = "gemma"
gpt_df["model"]   = "gpt3.5"

# Make sure both have the same schema; keep only needed columns
cols_to_keep = [
    "item_id",
    "prompt",
    "model",
    "readability",        # K–12 readability metric
    "morph_complexity",   # your morphological complexity metric
    "clarity",            # human rating dims
    "answer_acc",
    "distractor_quality",
    "word_diff_align",
    "task_diff_align",
]

df = pd.concat([gemma_df[cols_to_keep], gpt_df[cols_to_keep]], ignore_index=True)

##########################################
# 2. Descriptive stats by model/prompt   #
##########################################

metric_cols = [
    "readability",
    "morph_complexity",
    "clarity",
    "answer_acc",
    "distractor_quality",
    "word_diff_align",
    "task_diff_align",
]

group_cols = ["model", "prompt"]

summary = (
    df.groupby(group_cols)[metric_cols]
      .agg(["mean", "std", "count"])
      .reset_index()
)

print("Descriptive stats by model x prompt:")
print(summary)

####################################################
# 3. Reliability: Cronbach’s alpha for human dims  #
####################################################

def cronbach_alpha(df_scores: pd.DataFrame) -> float:
    """
    df_scores: DataFrame of shape (n_items, n_dims)
    rows = items; columns = rating dimensions (clarity, etc.)
    """
    # Drop rows with all NaNs
    X = df_scores.dropna(how="all").values
    k = X.shape[1]  # number of dimensions

    if k < 2:
        return np.nan

    # variance of each item (dimension)
    var_items = X.var(axis=0, ddof=1)
    # variance of total score
    total_scores = X.sum(axis=1)
    var_total = total_scores.var(ddof=1)

    alpha = (k / (k - 1)) * (1 - var_items.sum() / var_total)
    return alpha

human_dims = [
    "clarity",
    "answer_acc",
    "distractor_quality",
    "word_diff_align",
    "task_diff_align",
]

# Overall alpha across all items/models
alpha_overall = cronbach_alpha(df[human_dims])
print(f"\nCronbach's alpha (all items, all models): {alpha_overall:.3f}")

# Optional: alpha per model
for m in df["model"].unique():
    alpha_m = cronbach_alpha(df.loc[df["model"] == m, human_dims])
    print(f"Cronbach's alpha for {m}: {alpha_m:.3f}")

###########################################################
# 4. Significance: Gemma vs GPT-3.5 (paired by item+prompt)
###########################################################

# We assume: for each (item_id, prompt) we have one Gemma row and one GPT row.

def paired_t_tests(df, metric, model_col="model",
                   id_cols=("item_id", "prompt"),
                   model_a="gemma", model_b="gpt3.5"):
    """
    Paired t-tests between model_a and model_b on 'metric',
    pairing on id_cols (e.g., (item_id, prompt)).
    Returns a DataFrame of t-stat, p-value, and effect size per prompt.
    """
    results = []

    # do test separately for each prompt (more interpretable)
    for prompt in df["prompt"].unique():
        sub = df[df["prompt"] == prompt]

        # pivot: index = item_id (or item+prompt), columns = model, values = metric
        wide = (
            sub.pivot_table(index=list(id_cols), columns=model_col, values=metric)
               .dropna(subset=[model_a, model_b], how="any")
        )

        if wide.shape[0] < 3:
            # too few pairs for reliable t-test
            continue

        x = wide[model_a].values
        y = wide[model_b].values

        t_stat, p_val = stats.ttest_rel(x, y)

        # Cohen's d (paired)
        diff = x - y
        d = diff.mean() / diff.std(ddof=1)

        results.append(
            {
                "prompt": prompt,
                "metric": metric,
                "n_pairs": wide.shape[0],
                "t_stat": t_stat,
                "p_value": p_val,
                "cohens_d": d,
                "mean_" + model_a: x.mean(),
                "mean_" + model_b: y.mean(),
            }
        )

    return pd.DataFrame(results)


# Example: significance tests for K–12-relevant metrics
test_metrics = ["readability", "morph_complexity",
                "clarity", "answer_acc",
                "distractor_quality", "word_diff_align",
                "task_diff_align"]

all_tests = []
for m in test_metrics:
    res = paired_t_tests(df, metric=m)
    all_tests.append(res)

all_tests_df = pd.concat(all_tests, ignore_index=True)

print("\nPaired t-tests (Gemma vs GPT-3.5) by prompt & metric:")
print(all_tests_df.sort_values(["metric", "prompt"]))


import pandas as pd
import numpy as np
from scipy import stats

#############################
# 1. Load and harmonize data
#############################

# If your files are Excel:
#  - Gemma sheet name: "Gemma Results"
#  - GPT sheet name:   "Results"
gemma_raw = pd.read_excel("Gemma_Resultss.xlsx", sheet_name="Gemma Results")
gpt_raw   = pd.read_excel("Results_GPTa.xlsx",   sheet_name="Results")

print("Gemma columns:", gemma_raw.columns.tolist())
print("GPT columns:  ", gpt_raw.columns.tolist())

# Expected columns (based on what you wrote):
# Gemma: Prompting_strategy, Question_Type, ..., Overall_Score, Grammar_Score, Complexity_Score,
#        Readability_Score, Fluency_Score, Error_Count, Errors, ...
# GPT:   Row, Prompting_strategy, Question_Type, ..., Overall_Score, Grammar_Score, Complexity_Score,
#        Readability_Score, Fluency_Score, Error_Count, Errors

###############################################
# 2. Select & rename relevant columns per model
###############################################

# Common columns we care about
common_cols_gemma = [
    "Prompting_strategy",
    "Question_Type",
    "Overall_Score",
    "Grammar_Score",
    "Complexity_Score",
    "Readability_Score",
    "Fluency_Score",
]

common_cols_gpt = [
    "Prompting_strategy",
    "Question_Type",
    "Overall_Score",
    "Grammar_Score",
    "Complexity_Score",
    "Readability_Score",
    "Fluency_Score",
]

# Subset (this will raise a KeyError if any column is missing, so it's a good check)
gemma = gemma_raw[common_cols_gemma].copy()
gpt   = gpt_raw[common_cols_gpt].copy()

# Add model labels
gemma["model"] = "Gemma"
gpt["model"]   = "GPT-3.5"

# Normalize column names: Prompting_strategy -> prompt
gemma.rename(columns={"Prompting_strategy": "prompt"}, inplace=True)
gpt.rename(columns={"Prompting_strategy": "prompt"}, inplace=True)

# Also add lowercase metric aliases for convenience
rename_metrics = {
    "Overall_Score":     "overall",
    "Grammar_Score":     "grammar",
    "Complexity_Score":  "complexity",
    "Readability_Score": "readability",
    "Fluency_Score":     "fluency",
}

gemma = gemma.assign(**{new: gemma[old] for old, new in rename_metrics.items()})
gpt   = gpt.assign(**{new: gpt[old]   for old, new in rename_metrics.items()})

#########################################
# 3. Combine into a single long dataframe
#########################################

df = pd.concat([gemma, gpt], ignore_index=True)

# Clean up types (just to be safe)
metric_cols = list(rename_metrics.values())
for c in metric_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

print("\nCombined df head:")
print(df.head())

##############################################
# 4. Descriptive stats by model × prompt type
##############################################

# Descriptives: mean, std, count by model and prompting strategy
group_cols = ["model", "prompt"]
desc = (
    df.groupby(group_cols)[metric_cols]
      .agg(["mean", "std", "count"])
)

# Flatten MultiIndex columns: e.g. ('overall','mean') -> 'overall_mean'
desc.columns = [f"{m}_{stat}" for m, stat in desc.columns]
desc = desc.reset_index()

print("\nDescriptive statistics by model × prompting strategy:")
print(desc)

############################################
# 5. Global independent t-tests (Gemma vs GPT)
############################################

def global_t_tests(df, metrics, model_a="Gemma", model_b="GPT-3.5"):
    """Global (across all prompts) Welch t-tests for each metric."""
    rows = []
    for metric in metrics:
        a_vals = df.loc[df["model"] == model_a, metric].dropna().values
        b_vals = df.loc[df["model"] == model_b, metric].dropna().values

        if len(a_vals) < 2 or len(b_vals) < 2:
            # Not enough data for a reasonable t-test
            continue

        t_stat, p_val = stats.ttest_ind(a_vals, b_vals, equal_var=False)

        mean_a, mean_b = a_vals.mean(), b_vals.mean()
        sd_a, sd_b = a_vals.std(ddof=1), b_vals.std(ddof=1)
        n_a, n_b = len(a_vals), len(b_vals)

        # Cohen's d (pooled SD)
        pooled_sd = np.sqrt(((n_a - 1)*sd_a**2 + (n_b - 1)*sd_b**2) / (n_a + n_b - 2))
        cohens_d = (mean_a - mean_b) / pooled_sd if pooled_sd > 0 else np.nan

        rows.append(
            {
                "metric": metric,
                "n_gemma": n_a,
                "n_gpt": n_b,
                "mean_gemma": mean_a,
                "mean_gpt": mean_b,
                "t_stat": t_stat,
                "p_value": p_val,
                "cohens_d": cohens_d,
            }
        )

    return pd.DataFrame(rows)

global_tests = global_t_tests(df, metrics=metric_cols)
print("\nGlobal t-tests (Gemma vs GPT-3.5) across ALL prompts:")
print(global_tests)

##############################################################
# 6. Optional: per-prompt independent t-tests (robust version)
##############################################################

def independent_t_tests_by_prompt(df, metric, model_a="Gemma", model_b="GPT-3.5"):
    """Welch t-test for each prompt separately, for one metric."""
    rows = []
    for prompt in df["prompt"].unique():
        sub = df[df["prompt"] == prompt]

        a_vals = sub.loc[sub["model"] == model_a, metric].dropna().values
        b_vals = sub.loc[sub["model"] == model_b, metric].dropna().values

        # Require at least 2 items per group to be meaningful
        if len(a_vals) < 2 or len(b_vals) < 2:
            continue

        t_stat, p_val = stats.ttest_ind(a_vals, b_vals, equal_var=False)

        mean_a, mean_b = a_vals.mean(), b_vals.mean()
        sd_a, sd_b = a_vals.std(ddof=1), b_vals.std(ddof=1)
        n_a, n_b = len(a_vals), len(b_vals)
        pooled_sd = np.sqrt(((n_a - 1)*sd_a**2 + (n_b - 1)*sd_b**2) / (n_a + n_b - 2))
        cohens_d = (mean_a - mean_b) / pooled_sd if pooled_sd > 0 else np.nan

        rows.append(
            {
                "prompt": prompt,
                "metric": metric,
                "n_gemma": n_a,
                "n_gpt": n_b,
                "mean_gemma": mean_a,
                "mean_gpt": mean_b,
                "t_stat": t_stat,
                "p_value": p_val,
                "cohens_d": cohens_d,
            }
        )

    return pd.DataFrame(rows)

# Run for each metric and concatenate
per_prompt_results_list = []
for m in metric_cols:
    res_m = independent_t_tests_by_prompt(df, metric=m)
    print(f"Metric {m}: {len(res_m)} per-prompt rows")
    per_prompt_results_list.append(res_m)

if per_prompt_results_list:
    per_prompt_tests = pd.concat(per_prompt_results_list, ignore_index=True)
    print("\nPer-prompt t-tests (Gemma vs GPT-3.5):")
    print(per_prompt_tests.sort_values(["metric", "prompt"]))
else:
    per_prompt_tests = pd.DataFrame()
    print("\nNo per-prompt t-tests could be computed "
          "(likely < 2 items per model per prompt for each metric).")

###########################################
# 7. Save outputs (optional, for your paper)
###########################################

desc.to_csv("desc_by_model_prompt.csv", index=False)
global_tests.to_csv("global_t_tests_gemma_vs_gpt.csv", index=False)
per_prompt_tests.to_csv("per_prompt_t_tests_gemma_vs_gpt.csv", index=False)


In [13]:
#######################################
# 2. Standardize / rename columns     #
#######################################

# Gemma columns:
# Prompting_strategy, Question_Type, Question, Correct_Answer, Choice_1, Choice_2, Choice_3,
# Word_Difficulty, Task_Difficulty, Text, Overall_Score, Grammar_Score, Complexity_Score,
# Readability_Score, Fluency_Score, Error_Count, Errors, sum of scores

gemma_rename = {
    "Prompting_strategy": "prompt",
    "Question_Type": "question_type",
    "Word_Difficulty": "word_difficulty",
    "Task_Difficulty": "task_difficulty",
    "Text": "text",
    "Overall_Score": "overall",
    "Grammar_Score": "grammar",
    "Complexity_Score": "complexity",
    "Readability_Score": "readability",
    "Fluency_Score": "fluency",
    "sum of scores": "sum_scores",
}

gemma_df = gemma_df.rename(columns=gemma_rename)

# GPT columns:
# Row, Prompting_strategy, Question_Type, Question, Correct_Answer,
# Word_Difficulty, Task_Difficulty, Text, Overall_Score, Grammar_Score,
# Complexity_Score, Readability_Score, Fluency_Score, Error_Count, Errors

gpt_rename = {
    "Prompting_strategy": "prompt",
    "Question_Type": "question_type",
    "Word_Difficulty": "word_difficulty",
    "Task_Difficulty": "task_difficulty",
    "Text": "text",
    "Overall_Score": "overall",
    "Grammar_Score": "grammar",
    "Complexity_Score": "complexity",
    "Readability_Score": "readability",
    "Fluency_Score": "fluency",
}

gpt_df = gpt_df.rename(columns=gpt_rename)

# Add model label
gemma_df["model"] = "Gemma"
gpt_df["model"]   = "GPT-3.5"

# Create a simple item index within each (for reference only; not needed for t-tests)
gemma_df["item_id"] = np.arange(len(gemma_df))
gpt_df["item_id"]   = np.arange(len(gpt_df))


In [18]:
#########################################
# 3. Combine + descriptive statistics   #
#########################################

common_cols = [
    "item_id",
    "model",
    "prompt",
    "question_type",
    "word_difficulty",
    "task_difficulty",
    "overall",
    "grammar",
    "complexity",
    "readability",
    "fluency",
]

# Some cols (e.g., sum_scores) exist only in Gemma; we ignore them for now
df = pd.concat(
    [
        gemma_df[[c for c in common_cols if c in gemma_df.columns]],
        gpt_df[[c for c in common_cols if c in gpt_df.columns]],
    ],
    ignore_index=True,
)

metric_cols = ["overall", "grammar", "complexity", "readability", "fluency"]

# Descriptive stats by model × prompting strategy
desc_model_prompt = (
    df.groupby(["model", "prompt"])[metric_cols]
      .agg(["mean", "std", "count"])
      .reset_index()
)

print("\nDescriptive statistics by model × prompting strategy:")
print(desc_model_prompt)

# (Optional) also by model × question type:
desc_model_qtype = (
    df.groupby(["model", "question_type"])[metric_cols]
      .agg(["mean", "std", "count"])
      .reset_index()
)

print("\nDescriptive statistics by model × question type:")
print(desc_model_qtype)



Descriptive statistics by model × prompting strategy:
     model                               prompt    overall                   \
                                                      mean        std count   
0  GPT-3.5                     chain_of_thought  78.582774  13.160470   585   
1  GPT-3.5     chain_of_thought_plus_role_chain  78.308277  14.921074   585   
2  GPT-3.5     chain_of_thought_plus_sequential  78.538075  15.191198   584   
3  GPT-3.5  chain_of_thought_plus_sequential_rl  78.130354  13.463048   585   
4  GPT-3.5                             few_shot  81.950241  12.299584   585   
5  GPT-3.5                            zero_shot  79.419721  11.157753   585   

     grammar                  complexity                  readability  \
        mean        std count       mean        std count        mean   
0  94.518048  20.928433   585  92.461538  10.072969   585   78.442540   
1  91.927614  24.949208   585  91.025641  12.246373   585   82.233763   
2  92.693910  24.695

In [24]:
rename_metrics = {
    "Overall_Score":     "overall",
    "Grammar_Score":     "grammar",
    "Complexity_Score":  "complexity",
    "Readability_Score": "readability",
    "Fluency_Score":     "fluency",
    "Morph_Complexity":  "morph_complexity",   # new
    "Human_Total":       "human_total",        # new
}

import pandas as pd
import numpy as np
from scipy import stats

#############################
# 1. Load and harmonize data
#############################

# If your files are Excel:
#  - Gemma sheet name: "Gemma Results"
#  - GPT sheet name:   "Results"
gemma_raw = pd.read_excel("Gemma_Resultss.xlsx", sheet_name="Gemma Results")
gpt_raw   = pd.read_excel("Results_GPTa.xlsx",   sheet_name="Results")

print("Gemma columns:", gemma_raw.columns.tolist())
print("GPT columns:  ", gpt_raw.columns.tolist())

# Expected columns (based on what you wrote):
# Gemma: Prompting_strategy, Question_Type, ..., Overall_Score, Grammar_Score, Complexity_Score,
#        Readability_Score, Fluency_Score, Error_Count, Errors, ...
# GPT:   Row, Prompting_strategy, Question_Type, ..., Overall_Score, Grammar_Score, Complexity_Score,
#        Readability_Score, Fluency_Score, Error_Count, Errors

###############################################
# 2. Select & rename relevant columns per model
###############################################

# Common columns we care about
common_cols_gemma = [
    "Prompting_strategy",
    "Question_Type",
    "Overall_Score",
    "Grammar_Score",
    "Complexity_Score",
    "Readability_Score",
    "Fluency_Score",
]

common_cols_gpt = [
    "Prompting_strategy",
    "Question_Type",
    "Overall_Score",
    "Grammar_Score",
    "Complexity_Score",
    "Readability_Score",
    "Fluency_Score",
]

# Subset (this will raise a KeyError if any column is missing, so it's a good check)
gemma = gemma_raw[common_cols_gemma].copy()
gpt   = gpt_raw[common_cols_gpt].copy()

# Add model labels
gemma["model"] = "Gemma"
gpt["model"]   = "GPT-3.5"

# Normalize column names: Prompting_strategy -> prompt
gemma.rename(columns={"Prompting_strategy": "prompt"}, inplace=True)
gpt.rename(columns={"Prompting_strategy": "prompt"}, inplace=True)

# Also add lowercase metric aliases for convenience
rename_metrics = {
    "Overall_Score":     "overall",
    "Grammar_Score":     "grammar",
    "Complexity_Score":  "complexity",
    "Readability_Score": "readability",
    "Fluency_Score":     "fluency",
}

gemma = gemma.assign(**{new: gemma[old] for old, new in rename_metrics.items()})
gpt   = gpt.assign(**{new: gpt[old]   for old, new in rename_metrics.items()})

#########################################
# 3. Combine into a single long dataframe
#########################################

df = pd.concat([gemma, gpt], ignore_index=True)

# Clean up types (just to be safe)
metric_cols = list(rename_metrics.values())
for c in metric_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

print("\nCombined df head:")
print(df.head())

##############################################
# 4. Descriptive stats by model × prompt type
##############################################

# Descriptives: mean, std, count by model and prompting strategy
group_cols = ["model", "prompt"]
desc = (
    df.groupby(group_cols)[metric_cols]
      .agg(["mean", "std", "count"])
)

# Flatten MultiIndex columns: e.g. ('overall','mean') -> 'overall_mean'
desc.columns = [f"{m}_{stat}" for m, stat in desc.columns]
desc = desc.reset_index()

print("\nDescriptive statistics by model × prompting strategy:")
print(desc)

############################################
# 5. Global independent t-tests (Gemma vs GPT)
############################################

def global_t_tests(df, metrics, model_a="Gemma", model_b="GPT-3.5"):
    """Global (across all prompts) Welch t-tests for each metric."""
    rows = []
    for metric in metrics:
        a_vals = df.loc[df["model"] == model_a, metric].dropna().values
        b_vals = df.loc[df["model"] == model_b, metric].dropna().values

        if len(a_vals) < 2 or len(b_vals) < 2:
            # Not enough data for a reasonable t-test
            continue

        t_stat, p_val = stats.ttest_ind(a_vals, b_vals, equal_var=False)

        mean_a, mean_b = a_vals.mean(), b_vals.mean()
        sd_a, sd_b = a_vals.std(ddof=1), b_vals.std(ddof=1)
        n_a, n_b = len(a_vals), len(b_vals)

        # Cohen's d (pooled SD)
        pooled_sd = np.sqrt(((n_a - 1)*sd_a**2 + (n_b - 1)*sd_b**2) / (n_a + n_b - 2))
        cohens_d = (mean_a - mean_b) / pooled_sd if pooled_sd > 0 else np.nan

        rows.append(
            {
                "metric": metric,
                "n_gemma": n_a,
                "n_gpt": n_b,
                "mean_gemma": mean_a,
                "mean_gpt": mean_b,
                "t_stat": t_stat,
                "p_value": p_val,
                "cohens_d": cohens_d,
            }
        )

    return pd.DataFrame(rows)

global_tests = global_t_tests(df, metrics=metric_cols)
print("\nGlobal t-tests (Gemma vs GPT-3.5) across ALL prompts:")
print(global_tests)

##############################################################
# 6. Optional: per-prompt independent t-tests (robust version)
##############################################################

def independent_t_tests_by_prompt(df, metric, model_a="Gemma", model_b="GPT-3.5"):
    """Welch t-test for each prompt separately, for one metric."""
    rows = []
    for prompt in df["prompt"].unique():
        sub = df[df["prompt"] == prompt]

        a_vals = sub.loc[sub["model"] == model_a, metric].dropna().values
        b_vals = sub.loc[sub["model"] == model_b, metric].dropna().values

        # Require at least 2 items per group to be meaningful
        if len(a_vals) < 2 or len(b_vals) < 2:
            continue

        t_stat, p_val = stats.ttest_ind(a_vals, b_vals, equal_var=False)

        mean_a, mean_b = a_vals.mean(), b_vals.mean()
        sd_a, sd_b = a_vals.std(ddof=1), b_vals.std(ddof=1)
        n_a, n_b = len(a_vals), len(b_vals)
        pooled_sd = np.sqrt(((n_a - 1)*sd_a**2 + (n_b - 1)*sd_b**2) / (n_a + n_b - 2))
        cohens_d = (mean_a - mean_b) / pooled_sd if pooled_sd > 0 else np.nan

        rows.append(
            {
                "prompt": prompt,
                "metric": metric,
                "n_gemma": n_a,
                "n_gpt": n_b,
                "mean_gemma": mean_a,
                "mean_gpt": mean_b,
                "t_stat": t_stat,
                "p_value": p_val,
                "cohens_d": cohens_d,
            }
        )

    return pd.DataFrame(rows)

# Run for each metric and concatenate
per_prompt_results_list = []
for m in metric_cols:
    res_m = independent_t_tests_by_prompt(df, metric=m)
    print(f"Metric {m}: {len(res_m)} per-prompt rows")
    per_prompt_results_list.append(res_m)

if per_prompt_results_list:
    per_prompt_tests = pd.concat(per_prompt_results_list, ignore_index=True)
    print("\nPer-prompt t-tests (Gemma vs GPT-3.5):")
    print(per_prompt_tests.sort_values(["metric", "prompt"]))
else:
    per_prompt_tests = pd.DataFrame()
    print("\nNo per-prompt t-tests could be computed "
          "(likely < 2 items per model per prompt for each metric).")

###########################################
# 7. Save outputs (optional, for your paper)
###########################################

desc.to_csv("desc_by_model_prompt.csv", index=False)
global_tests.to_csv("global_t_tests_gemma_vs_gpt.csv", index=False)
per_prompt_tests.to_csv("per_prompt_t_tests_gemma_vs_gpt.csv", index=False)


In [30]:
import pandas as pd
import numpy as np
from scipy import stats

#############################
# 1. Load and harmonize data
#############################

# Files and sheet names you specified
gemma_raw = pd.read_excel("Gemma_Resultss.xlsx", sheet_name="Gemma Results")
gpt_raw   = pd.read_excel("Results_GPTa.xlsx",   sheet_name="Results")

print("Gemma columns:", gemma_raw.columns.tolist())
print("GPT columns:  ", gpt_raw.columns.tolist())

###############################################
# 2. Select & rename relevant columns per model
###############################################

# Columns we care about from each file
common_cols_gemma = [
    "Prompting_strategy",
    "Question_Type",
    "Overall_Score",
    "Grammar_Score",
    "Complexity_Score",
    "Readability_Score",
    "Fluency_Score",
]

common_cols_gpt = [
    "Prompting_strategy",
    "Question_Type",
    "Overall_Score",
    "Grammar_Score",
    "Complexity_Score",
    "Readability_Score",
    "Fluency_Score",
]

# Subset safely (will raise if a column is missing → good sanity check)
gemma = gemma_raw[common_cols_gemma].copy()
gpt   = gpt_raw[common_cols_gpt].copy()

# Add model labels
gemma["model"] = "Gemma"
gpt["model"]   = "GPT-3.5"

# Normalize prompt column name
gemma.rename(columns={"Prompting_strategy": "prompt"}, inplace=True)
gpt.rename(columns={"Prompting_strategy": "prompt"}, inplace=True)

# Create lowercase metric aliases (easier to work with)
rename_metrics = {
    "Overall_Score":     "overall",
    "Grammar_Score":     "grammar",
    "Complexity_Score":  "complexity",
    "Readability_Score": "readability",
    "Fluency_Score":     "fluency",
}

for old, new in rename_metrics.items():
    gemma[new] = gemma[old]
    gpt[new]   = gpt[old]

metric_cols = list(rename_metrics.values())

#########################################
# 3. Combine into a single long dataframe
#########################################

df = pd.concat([gemma, gpt], ignore_index=True)

# Clean numeric types
for c in metric_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

print("\nCombined df head:")
print(df.head())

##############################################
# 4. Descriptive stats by model × prompt type
##############################################

group_cols = ["model", "prompt"]
desc = (
    df.groupby(group_cols)[metric_cols]
      .agg(["mean", "std", "count"])
)

# Flatten MultiIndex columns
desc.columns = [f"{m}_{stat}" for m, stat in desc.columns]
desc = desc.reset_index()

print("\nDescriptive statistics by model × prompting strategy:")
print(desc)

############################################
# 5. Global independent t-tests (Gemma vs GPT)
############################################

def global_t_tests(df, metrics, model_a="Gemma", model_b="GPT-3.5"):
    """Global (across all prompts) Welch t-tests for each metric."""
    rows = []
    for metric in metrics:
        a_vals = df.loc[df["model"] == model_a, metric].dropna().values
        b_vals = df.loc[df["model"] == model_b, metric].dropna().values

        if len(a_vals) < 2 or len(b_vals) < 2:
            continue

        t_stat, p_val = stats.ttest_ind(a_vals, b_vals, equal_var=False)

        mean_a, mean_b = a_vals.mean(), b_vals.mean()
        sd_a, sd_b = a_vals.std(ddof=1), b_vals.std(ddof=1)
        n_a, n_b = len(a_vals), len(b_vals)

        pooled_sd = np.sqrt(((n_a - 1)*sd_a**2 + (n_b - 1)*sd_b**2) / (n_a + n_b - 2))
        cohens_d = (mean_a - mean_b) / pooled_sd if pooled_sd > 0 else np.nan

        rows.append(
            {
                "metric": metric,
                "n_gemma": n_a,
                "n_gpt": n_b,
                "mean_gemma": mean_a,
                "mean_gpt": mean_b,
                "t_stat": t_stat,
                "p_value": p_val,
                "cohens_d": cohens_d,
            }
        )

    return pd.DataFrame(rows)

global_tests = global_t_tests(df, metrics=metric_cols)
print("\nGlobal t-tests (Gemma vs GPT-3.5) across ALL prompts:")
print(global_tests)

##############################################################
# 6. Per-prompt independent t-tests (with safe handling)
##############################################################

def independent_t_tests_by_prompt(df, metric, model_a="Gemma", model_b="GPT-3.5"):
    """Welch t-test for each prompt separately, for one metric."""
    rows = []
    for prompt in df["prompt"].unique():
        sub = df[df["prompt"] == prompt]

        a_vals = sub.loc[sub["model"] == model_a, metric].dropna().values
        b_vals = sub.loc[sub["model"] == model_b, metric].dropna().values

        # Need at least 2 per group for a meaningful test
        if len(a_vals) < 2 or len(b_vals) < 2:
            continue

        t_stat, p_val = stats.ttest_ind(a_vals, b_vals, equal_var=False)

        mean_a, mean_b = a_vals.mean(), b_vals.mean()
        sd_a, sd_b = a_vals.std(ddof=1), b_vals.std(ddof=1)
        n_a, n_b = len(a_vals), len(b_vals)

        pooled_sd = np.sqrt(((n_a - 1)*sd_a**2 + (n_b - 1)*sd_b**2) / (n_a + n_b - 2))
        cohens_d = (mean_a - mean_b) / pooled_sd if pooled_sd > 0 else np.nan

        rows.append(
            {
                "prompt": prompt,
                "metric": metric,
                "n_gemma": n_a,
                "n_gpt": n_b,
                "mean_gemma": mean_a,
                "mean_gpt": mean_b,
                "t_stat": t_stat,
                "p_value": p_val,
                "cohens_d": cohens_d,
            }
        )

    return pd.DataFrame(rows)

per_prompt_results_list = []
for m in metric_cols:
    res_m = independent_t_tests_by_prompt(df, metric=m)
    print(f"Metric {m}: {len(res_m)} per-prompt rows")
    if not res_m.empty:
        per_prompt_results_list.append(res_m)

if per_prompt_results_list:
    per_prompt_tests = pd.concat(per_prompt_results_list, ignore_index=True)
    print("\nPer-prompt t-tests (Gemma vs GPT-3.5):")
    print("Columns:", per_prompt_tests.columns.tolist())

    # Only sort by columns that actually exist → avoids KeyError
    sort_cols = [c for c in ["metric", "prompt"] if c in per_prompt_tests.columns]
    if sort_cols:
        print(per_prompt_tests.sort_values(sort_cols))
    else:
        print(per_prompt_tests)
else:
    per_prompt_tests = pd.DataFrame()
    print("\nNo per-prompt t-tests could be computed "
          "(likely < 2 items per model per prompt for each metric).")

###########################################
# 7. Save outputs (optional)
###########################################

desc.to_csv("desc_by_model_prompt.csv", index=False)
global_tests.to_csv("global_t_tests_gemma_vs_gpt.csv", index=False)
per_prompt_tests.to_csv("per_prompt_t_tests_gemma_vs_gpt.csv", index=False)


Gemma columns: ['Prompting_strategy', 'Question_Type', 'Question', 'Correct_Answer', 'Choice_1', 'Choice_2', 'Choice_3', 'Word_Difficulty', 'Task_Difficulty', 'Text', 'Overall_Score', 'Grammar_Score', 'Complexity_Score', 'Readability_Score', 'Fluency_Score', 'Error_Count', 'Errors', 'Unnamed: 17']
GPT columns:   ['Row', 'Prompting_strategy', 'Question_Type', 'Question', 'Correct_Answer', 'Word_Difficulty', 'Task_Difficulty', 'Text', 'Overall_Score', 'Grammar_Score', 'Complexity_Score', 'Readability_Score', 'Fluency_Score', 'Error_Count', 'Errors']

Combined df head:
       prompt  Question_Type  Overall_Score  Grammar_Score  Complexity_Score  \
0  CoT+Seq_Rl            1.0         73.589          100.0              80.0   
1  CoT+Seq_Rl            1.0         67.949          100.0              80.0   
2  CoT+Seq_Rl            1.0         65.129          100.0              80.0   
3  CoT+Seq_Rl            1.0         70.769          100.0              80.0   
4  CoT+Seq_Rl            1.

In [44]:
import pandas as pd
import numpy as np
from scipy import stats

#############################
# 1. Load and harmonize data
#############################

# Files and sheet names you specified
gemma_raw = pd.read_excel("Gemma_Resultss.xlsx", sheet_name="Gemma Results")
gpt_raw   = pd.read_excel("Results_GPTa.xlsx",   sheet_name="Results")

print("Gemma columns:", gemma_raw.columns.tolist())
print("GPT columns:  ", gpt_raw.columns.tolist())

###############################################
# 2. Select & rename relevant columns per model
###############################################

# Columns we care about from each file
common_cols_gemma = [
    "Prompting_strategy",
    "Question_Type",
    "Overall_Score",
    "Grammar_Score",
    "Complexity_Score",
    "Readability_Score",
    "Fluency_Score",
]

common_cols_gpt = [
    "Prompting_strategy",
    "Question_Type",
    "Overall_Score",
    "Grammar_Score",
    "Complexity_Score",
    "Readability_Score",
    "Fluency_Score",
]

# Subset safely (will raise if a column is missing → good sanity check)
gemma = gemma_raw[common_cols_gemma].copy()
gpt   = gpt_raw[common_cols_gpt].copy()

# Add model labels
gemma["model"] = "Gemma"
gpt["model"]   = "GPT-3.5"

# Normalize prompt column name
gemma.rename(columns={"Prompting_strategy": "prompt"}, inplace=True)
gpt.rename(columns={"Prompting_strategy": "prompt"}, inplace=True)

# Create lowercase metric aliases (easier to work with)
rename_metrics = {
    "Overall_Score":     "overall",
    "Grammar_Score":     "grammar",
    "Complexity_Score":  "complexity",
    "Readability_Score": "readability",
    "Fluency_Score":     "fluency",
}

for old, new in rename_metrics.items():
    gemma[new] = gemma[old]
    gpt[new]   = gpt[old]

metric_cols = list(rename_metrics.values())

#########################################
# 3. Combine into a single long dataframe
#########################################

df = pd.concat([gemma, gpt], ignore_index=True)

# Clean numeric types
for c in metric_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

print("\nCombined df head:")
print(df.head())

##############################################
# 4. Descriptive stats by model × prompt type
##############################################

group_cols = ["model", "prompt"]
desc = (
    df.groupby(group_cols)[metric_cols]
      .agg(["mean", "std", "count"])
)

# Flatten MultiIndex columns
desc.columns = [f"{m}_{stat}" for m, stat in desc.columns]
desc = desc.reset_index()

print("\nDescriptive statistics by model × prompting strategy:")
print(desc)

############################################
# 5. Global independent t-tests (Gemma vs GPT)
############################################

def global_t_tests(df, metrics, model_a="Gemma", model_b="GPT-3.5"):
    """Global (across all prompts) Welch t-tests for each metric."""
    rows = []
    for metric in metrics:
        a_vals = df.loc[df["model"] == model_a, metric].dropna().values
        b_vals = df.loc[df["model"] == model_b, metric].dropna().values

        if len(a_vals) < 2 or len(b_vals) < 2:
            continue

        t_stat, p_val = stats.ttest_ind(a_vals, b_vals, equal_var=False)

        mean_a, mean_b = a_vals.mean(), b_vals.mean()
        sd_a, sd_b = a_vals.std(ddof=1), b_vals.std(ddof=1)
        n_a, n_b = len(a_vals), len(b_vals)

        pooled_sd = np.sqrt(((n_a - 1)*sd_a**2 + (n_b - 1)*sd_b**2) / (n_a + n_b - 2))
        cohens_d = (mean_a - mean_b) / pooled_sd if pooled_sd > 0 else np.nan

        rows.append(
            {
                "metric": metric,
                "n_gemma": n_a,
                "n_gpt": n_b,
                "mean_gemma": mean_a,
                "mean_gpt": mean_b,
                "t_stat": t_stat,
                "p_value": p_val,
                "cohens_d": cohens_d,
            }
        )

    return pd.DataFrame(rows)

global_tests = global_t_tests(df, metrics=metric_cols)
print("\nGlobal t-tests (Gemma vs GPT-3.5) across ALL prompts:")
print(global_tests)

##############################################################
# 6. Per-prompt independent t-tests (with safe handling)
##############################################################

def independent_t_tests_by_prompt(df, metric, model_a="Gemma", model_b="GPT-3.5"):
    """Welch t-test for each prompt separately, for one metric."""
    rows = []
    for prompt in df["prompt"].unique():
        sub = df[df["prompt"] == prompt]

        a_vals = sub.loc[sub["model"] == model_a, metric].dropna().values
        b_vals = sub.loc[sub["model"] == model_b, metric].dropna().values

        # Need at least 2 per group for a meaningful test
        if len(a_vals) < 2 or len(b_vals) < 2:
            continue

        t_stat, p_val = stats.ttest_ind(a_vals, b_vals, equal_var=False)

        mean_a, mean_b = a_vals.mean(), b_vals.mean()
        sd_a, sd_b = a_vals.std(ddof=1), b_vals.std(ddof=1)
        n_a, n_b = len(a_vals), len(b_vals)

        pooled_sd = np.sqrt(((n_a - 1)*sd_a**2 + (n_b - 1)*sd_b**2) / (n_a + n_b - 2))
        cohens_d = (mean_a - mean_b) / pooled_sd if pooled_sd > 0 else np.nan

        rows.append(
            {
                "prompt": prompt,
                "metric": metric,
                "n_gemma": n_a,
                "n_gpt": n_b,
                "mean_gemma": mean_a,
                "mean_gpt": mean_b,
                "t_stat": t_stat,
                "p_value": p_val,
                "cohens_d": cohens_d,
            }
        )

    return pd.DataFrame(rows)

per_prompt_results_list = []
for m in metric_cols:
    res_m = independent_t_tests_by_prompt(df, metric=m)
    print(f"Metric {m}: {len(res_m)} per-prompt rows")
    if not res_m.empty:
        per_prompt_results_list.append(res_m)

if per_prompt_results_list:
    per_prompt_tests = pd.concat(per_prompt_results_list, ignore_index=True)
    print("\nPer-prompt t-tests (Gemma vs GPT-3.5):")
    print("Columns:", per_prompt_tests.columns.tolist())

    # Only sort by columns that actually exist → avoids KeyError
    sort_cols = [c for c in ["metric", "prompt"] if c in per_prompt_tests.columns]
    if sort_cols:
        print(per_prompt_tests.sort_values(sort_cols))
    else:
        print(per_prompt_tests)
else:
    per_prompt_tests = pd.DataFrame()
    print("\nNo per-prompt t-tests could be computed "
          "(likely < 2 items per model per prompt for each metric).")

###########################################################
# 7. Cross-prompt pairwise t-tests (Gemma vs GPT-3.5)
###########################################################

def compare_prompt_pairs(df, 
                         model1, prompt1, 
                         model2, prompt2, 
                         metric):
    """
    Compare Model1-Prompt1 vs Model2-Prompt2 on a specific metric.
    Uses Welch's t-test and computes Cohen's d.
    Returns a dict with results (or None if insufficient data).
    """
    mask1 = (df["model"] == model1) & (df["prompt"] == prompt1)
    mask2 = (df["model"] == model2) & (df["prompt"] == prompt2)

    vals1 = df.loc[mask1, metric].dropna().values
    vals2 = df.loc[mask2, metric].dropna().values

    if len(vals1) < 2 or len(vals2) < 2:
        print(f"\n[SKIP] Not enough data for {model1}-{prompt1} vs {model2}-{prompt2} on {metric}")
        return None

    t_stat, p_val = stats.ttest_ind(vals1, vals2, equal_var=False)

    mean1, mean2 = vals1.mean(), vals2.mean()
    sd1, sd2 = vals1.std(ddof=1), vals2.std(ddof=1)
    n1, n2 = len(vals1), len(vals2)

    pooled_sd = np.sqrt(((n1 - 1)*sd1**2 + (n2 - 1)*sd2**2) / (n1 + n2 - 2))
    cohens_d = (mean1 - mean2) / pooled_sd if pooled_sd > 0 else np.nan

    print(f"\n### Comparison: {model1}-{prompt1}  vs  {model2}-{prompt2}")
    print(f"Metric: {metric}")
    print(f"N1={n1}, Mean1={mean1:.2f}")
    print(f"N2={n2}, Mean2={mean2:.2f}")
    print(f"T-stat={t_stat:.4f}, p-value={p_val:.6f}, Cohen's d={cohens_d:.3f}")

    if p_val >= 0.05:
        print("➡ The second combo does *NOT* significantly outperform the first (p ≥ 0.05).")

    return {
        "model1": model1,
        "prompt1": prompt1,
        "model2": model2,
        "prompt2": prompt2,
        "metric": metric,
        "n1": n1,
        "n2": n2,
        "mean1": mean1,
        "mean2": mean2,
        "t_stat": t_stat,
        "p_value": p_val,
        "cohens_d": cohens_d,
    }




# ------------------------------------------------------------------
# Define the specific cross-prompt comparisons you care about
# (edit this list as needed based on your actual prompt labels)
# ------------------------------------------------------------------
cross_prompt_specs = [
    # Example: Gemma CoT vs GPT-3.5 zero_shot (overall)
    {"model1": "Gemma",   "prompt1": "CoT",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought"},
    
    # Example: Gemma CoT+Seq vs GPT-3.5 chain_of_thought_plus_sequential
    {"model1": "Gemma",   "prompt1": "CoT",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_role_chain"},
    
    # Example: Gemma Few_Shot vs GPT-3.5 few_shot
    {"model1": "Gemma",   "prompt1": "CoT",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_sequential"},

    {"model1": "Gemma",   "prompt1": "CoT",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_sequential_rl"},

    {"model1": "Gemma",   "prompt1": "CoT",
     "model2": "GPT-3.5", "prompt2": "few_shot"},
    
    {"model1": "Gemma",   "prompt1": "CoT",
     "model2": "GPT-3.5", "prompt2": "zero_shot"},
    
    # Add more pairs as you like...

    {"model1": "Gemma",   "prompt1": "CoT+RC",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought"},
    
    # Example: Gemma CoT+Seq vs GPT-3.5 chain_of_thought_plus_sequential
    {"model1": "Gemma",   "prompt1": "CoT+RC",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_role_chain"},
    
    # Example: Gemma Few_Shot vs GPT-3.5 few_shot
    {"model1": "Gemma",   "prompt1": "CoT+RC",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_sequential"},

    {"model1": "Gemma",   "prompt1": "CoT+RC",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_sequential_rl"},

    {"model1": "Gemma",   "prompt1": "CoT+RC",
     "model2": "GPT-3.5", "prompt2": "few_shot"},
    
    {"model1": "Gemma",   "prompt1": "CoT+RC",
     "model2": "GPT-3.5", "prompt2": "zero_shot"},


  #
    {"model1": "Gemma",   "prompt1": "CoT+Seq",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought"},
    
    # Example: Gemma CoT+Seq vs GPT-3.5 chain_of_thought_plus_sequential
    {"model1": "Gemma",   "prompt1": "CoT+Seq",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_role_chain"},
    
    # Example: Gemma Few_Shot vs GPT-3.5 few_shot
    {"model1": "Gemma",   "prompt1": "CoT+Seq",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_sequential"},

    {"model1": "Gemma",   "prompt1": "CoT+Seq",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_sequential_rl"},

    {"model1": "Gemma",   "prompt1": "CoT+Seq",
     "model2": "GPT-3.5", "prompt2": "few_shot"},
    
    {"model1": "Gemma",   "prompt1": "CoT+Seq",
     "model2": "GPT-3.5", "prompt2": "zero_shot"},

#
    {"model1": "Gemma",   "prompt1": "CoT+Seq_Rl",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought"},
    
    # Example: Gemma CoT+Seq vs GPT-3.5 chain_of_thought_plus_sequential
    {"model1": "Gemma",   "prompt1": "CoT+Seq_Rl",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_role_chain"},
    
    # Example: Gemma Few_Shot vs GPT-3.5 few_shot
    {"model1": "Gemma",   "prompt1": "CoT+Seq_Rl",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_sequential"},

    {"model1": "Gemma",   "prompt1": "CoT+Seq_Rl",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_sequential_rl"},

    {"model1": "Gemma",   "prompt1": "CoT+Seq_Rl",
     "model2": "GPT-3.5", "prompt2": "few_shot"},
    
    {"model1": "Gemma",   "prompt1": "CoT+Seq_Rl",
     "model2": "GPT-3.5", "prompt2": "zero_shot"},

#
    {"model1": "Gemma",   "prompt1": "Few_Shot",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought"},
    
    # Example: Gemma CoT+Seq vs GPT-3.5 chain_of_thought_plus_sequential
    {"model1": "Gemma",   "prompt1": "Few_Shot",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_role_chain"},
    
    # Example: Gemma Few_Shot vs GPT-3.5 few_shot
    {"model1": "Gemma",   "prompt1": "Few_Shot",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_sequential"},

    {"model1": "Gemma",   "prompt1": "Few_Shot",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_sequential_rl"},

    {"model1": "Gemma",   "prompt1": "Few_Shot",
     "model2": "GPT-3.5", "prompt2": "few_shot"},
    
    {"model1": "Gemma",   "prompt1": "Few_Shot",
     "model2": "GPT-3.5", "prompt2": "zero_shot"},

    {"model1": "Gemma",   "prompt1": "Zero_Shot",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought"},
    
    # Example: Gemma CoT+Seq vs GPT-3.5 chain_of_thought_plus_sequential
    {"model1": "Gemma",   "prompt1": "Zero_Shot",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_role_chain"},
    
    # Example: Gemma Few_Shot vs GPT-3.5 few_shot
    {"model1": "Gemma",   "prompt1": "Zero_Shot",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_sequential"},

    {"model1": "Gemma",   "prompt1": "Zero_Shot",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_sequential_rl"},

    {"model1": "Gemma",   "prompt1": "Zero_Shot",
     "model2": "GPT-3.5", "prompt2": "few_shot"},
    
    {"model1": "Gemma",   "prompt1": "Zero_Shot",
     "model2": "GPT-3.5", "prompt2": "zero_shot"},

]

metrics_to_compare = ["overall", "grammar", "complexity", "readability", "fluency"]

cross_prompt_rows = []

for spec in cross_prompt_specs:
    for m in metrics_to_compare:
        res = compare_prompt_pairs(
            df,
            model1=spec["model1"],
            prompt1=spec["prompt1"],
            model2=spec["model2"],
            prompt2=spec["prompt2"],
            metric=m
        )
        if res is not None:
            cross_prompt_rows.append(res)

if cross_prompt_rows:
    cross_prompt_tests = pd.DataFrame(cross_prompt_rows)
    print("\nCross-prompt pairwise t-tests (Gemma vs GPT-3.5):")
    print(cross_prompt_tests)
else:
    cross_prompt_tests = pd.DataFrame()
    print("\nNo cross-prompt tests could be computed (check prompt labels / sample sizes).")

###########################################
# 8. Save outputs (optional)
###########################################

desc.to_csv("desc_by_model_prompt.csv", index=False)
global_tests.to_csv("global_t_tests_gemma_vs_gpt.csv", index=False)
per_prompt_tests.to_csv("per_prompt_t_tests_gemma_vs_gpt.csv", index=False)
cross_prompt_tests.to_csv("cross_prompt_pairwise_t_tests_gemma_vs_gpt.csv", index=False)


Gemma columns: ['Prompting_strategy', 'Question_Type', 'Question', 'Correct_Answer', 'Choice_1', 'Choice_2', 'Choice_3', 'Word_Difficulty', 'Task_Difficulty', 'Text', 'Overall_Score', 'Grammar_Score', 'Complexity_Score', 'Readability_Score', 'Fluency_Score', 'Error_Count', 'Errors', 'Unnamed: 17']
GPT columns:   ['Row', 'Prompting_strategy', 'Question_Type', 'Question', 'Correct_Answer', 'Word_Difficulty', 'Task_Difficulty', 'Text', 'Overall_Score', 'Grammar_Score', 'Complexity_Score', 'Readability_Score', 'Fluency_Score', 'Error_Count', 'Errors']

Combined df head:
       prompt  Question_Type  Overall_Score  Grammar_Score  Complexity_Score  \
0  CoT+Seq_Rl            1.0         73.589          100.0              80.0   
1  CoT+Seq_Rl            1.0         67.949          100.0              80.0   
2  CoT+Seq_Rl            1.0         65.129          100.0              80.0   
3  CoT+Seq_Rl            1.0         70.769          100.0              80.0   
4  CoT+Seq_Rl            1.

In [46]:
import pandas as pd
import numpy as np
from scipy import stats

#############################
# 1. Load and harmonize data
#############################

# Files and sheet names you specified
gemma_raw = pd.read_excel("Gemma_Resultss.xlsx", sheet_name="Gemma Results")
gpt_raw   = pd.read_excel("Results_GPTa.xlsx",   sheet_name="Results")

print("Gemma columns:", gemma_raw.columns.tolist())
print("GPT columns:  ", gpt_raw.columns.tolist())

###############################################
# 2. Select & rename relevant columns per model
###############################################

# Columns we care about from each file
common_cols_gemma = [
    "Prompting_strategy",
    "Question_Type",
    "Overall_Score",
    "Grammar_Score",
    "Complexity_Score",
    "Readability_Score",
    "Fluency_Score",
]

common_cols_gpt = [
    "Prompting_strategy",
    "Question_Type",
    "Overall_Score",
    "Grammar_Score",
    "Complexity_Score",
    "Readability_Score",
    "Fluency_Score",
]

# Subset safely (will raise if a column is missing → good sanity check)
gemma = gemma_raw[common_cols_gemma].copy()
gpt   = gpt_raw[common_cols_gpt].copy()

# Add model labels
gemma["model"] = "Gemma"
gpt["model"]   = "GPT-3.5"

# Normalize prompt column name
gemma.rename(columns={"Prompting_strategy": "prompt"}, inplace=True)
gpt.rename(columns={"Prompting_strategy": "prompt"}, inplace=True)

# Create lowercase metric aliases (easier to work with)
rename_metrics = {
    "Overall_Score":     "overall",
    "Grammar_Score":     "grammar",
    "Complexity_Score":  "complexity",
    "Readability_Score": "readability",
    "Fluency_Score":     "fluency",
}

for old, new in rename_metrics.items():
    gemma[new] = gemma[old]
    gpt[new]   = gpt[old]

metric_cols = list(rename_metrics.values())

#########################################
# 3. Combine into a single long dataframe
#########################################

df = pd.concat([gemma, gpt], ignore_index=True)

# Clean numeric types
for c in metric_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

print("\nCombined df head:")
print(df.head())

##############################################
# 4. Descriptive stats by model × prompt type
##############################################

group_cols = ["model", "prompt"]
desc = (
    df.groupby(group_cols)[metric_cols]
      .agg(["mean", "std", "count"])
)

# Flatten MultiIndex columns
desc.columns = [f"{m}_{stat}" for m, stat in desc.columns]
desc = desc.reset_index()

print("\nDescriptive statistics by model × prompting strategy:")
print(desc)

############################################
# 5. Global independent t-tests (Gemma vs GPT)
############################################

def global_t_tests(df, metrics, model_a="Gemma", model_b="GPT-3.5"):
    """Global (across all prompts) Welch t-tests for each metric."""
    rows = []
    for metric in metrics:
        a_vals = df.loc[df["model"] == model_a, metric].dropna().values
        b_vals = df.loc[df["model"] == model_b, metric].dropna().values

        if len(a_vals) < 2 or len(b_vals) < 2:
            continue

        # Welch t-test
        t_stat, p_val = stats.ttest_ind(a_vals, b_vals, equal_var=False)

        mean_a, mean_b = a_vals.mean(), b_vals.mean()
        sd_a, sd_b = a_vals.std(ddof=1), b_vals.std(ddof=1)
        n_a, n_b = len(a_vals), len(b_vals)

        pooled_sd = np.sqrt(((n_a - 1)*sd_a**2 + (n_b - 1)*sd_b**2) / (n_a + n_b - 2))
        cohens_d = (mean_a - mean_b) / pooled_sd if pooled_sd > 0 else np.nan

        rows.append(
            {
                "metric": metric,
                "n_gemma": n_a,
                "n_gpt": n_b,
                "mean_gemma": mean_a,
                "mean_gpt": mean_b,
                "t_stat": t_stat,
                "p_value": p_val,
                "cohens_d": cohens_d,
            }
        )

    return pd.DataFrame(rows)

global_tests = global_t_tests(df, metrics=metric_cols)
print("\nGlobal t-tests (Gemma vs GPT-3.5) across ALL prompts:")
print(global_tests)

##############################################################
# 6. Per-prompt independent t-tests (same prompt name in both)
##############################################################

def independent_t_tests_by_prompt(df, metric, model_a="Gemma", model_b="GPT-3.5"):
    """
    Welch t-test for each *prompt name* separately, for one metric,
    assuming both models share the same prompt label.
    """
    rows = []
    for prompt in df["prompt"].unique():
        sub = df[df["prompt"] == prompt]

        a_vals = sub.loc[sub["model"] == model_a, metric].dropna().values
        b_vals = sub.loc[sub["model"] == model_b, metric].dropna().values

        # Need at least 2 per group for a meaningful test
        if len(a_vals) < 2 or len(b_vals) < 2:
            continue

        t_stat, p_val = stats.ttest_ind(a_vals, b_vals, equal_var=False)

        mean_a, mean_b = a_vals.mean(), b_vals.mean()
        sd_a, sd_b = a_vals.std(ddof=1), b_vals.std(ddof=1)
        n_a, n_b = len(a_vals), len(b_vals)

        pooled_sd = np.sqrt(((n_a - 1)*sd_a**2 + (n_b - 1)*sd_b**2) / (n_a + n_b - 2))
        cohens_d = (mean_a - mean_b) / pooled_sd if pooled_sd > 0 else np.nan

        rows.append(
            {
                "prompt": prompt,
                "metric": metric,
                "n_gemma": n_a,
                "n_gpt": n_b,
                "mean_gemma": mean_a,
                "mean_gpt": mean_b,
                "t_stat": t_stat,
                "p_value": p_val,
                "cohens_d": cohens_d,
            }
        )

    return pd.DataFrame(rows)

per_prompt_results_list = []
for m in metric_cols:
    res_m = independent_t_tests_by_prompt(df, metric=m)
    print(f"Metric {m}: {len(res_m)} per-prompt rows")
    if not res_m.empty:
        per_prompt_results_list.append(res_m)

if per_prompt_results_list:
    per_prompt_tests = pd.concat(per_prompt_results_list, ignore_index=True)
    print("\nPer-prompt t-tests (Gemma vs GPT-3.5), same prompt name:")
    sort_cols = [c for c in ["metric", "prompt"] if c in per_prompt_tests.columns]
    if sort_cols:
        print(per_prompt_tests.sort_values(sort_cols))
    else:
        print(per_prompt_tests)
else:
    per_prompt_tests = pd.DataFrame()
    print("\nNo per-prompt t-tests could be computed "
          "(likely < 2 items per model per prompt for each metric).")

###################################################################
# 7. Matched prompt-pair t-tests (your CoT ↔ chain_of_thought etc.)
###################################################################

# Mapping: Gemma prompting → GPT prompting
prompt_pairs = {
    "CoT":        "chain_of_thought",
    "CoT+RC":     "chain_of_thought_plus_role_chain",
    "CoT+Seq":    "chain_of_thought_plus_sequential",
    "CoT+Seq_Rl": "chain_of_thought_plus_sequential_rl",
    "Few_Shot":   "few_shot",
    "Zero_Shot":  "zero_shot",
}

def compare_prompt_pairs(df, metrics, prompt_pairs,
                         model_a="Gemma", model_b="GPT-3.5"):
    """
    For each (Gemma_prompt, GPT_prompt) pair and each metric, run a Welch t-test.
    This is what you use to say:
    'GPT with X prompting does NOT significantly outperform Gemma with Y prompting.'
    """
    rows = []
    for gemma_prompt, gpt_prompt in prompt_pairs.items():
        for metric in metrics:
            # Extract values
            a_vals = df.loc[
                (df["model"] == model_a) & (df["prompt"] == gemma_prompt),
                metric
            ].dropna().values

            b_vals = df.loc[
                (df["model"] == model_b) & (df["prompt"] == gpt_prompt),
                metric
            ].dropna().values

            if len(a_vals) < 2 or len(b_vals) < 2:
                # Not enough data for this comparison
                continue

            t_stat, p_val = stats.ttest_ind(a_vals, b_vals, equal_var=False)

            mean_a, mean_b = a_vals.mean(), b_vals.mean()
            sd_a, sd_b = a_vals.std(ddof=1), b_vals.std(ddof=1)
            n_a, n_b = len(a_vals), len(b_vals)

            pooled_sd = np.sqrt(((n_a - 1)*sd_a**2 + (n_b - 1)*sd_b**2) /
                                (n_a + n_b - 2))
            cohens_d = (mean_a - mean_b) / pooled_sd if pooled_sd > 0 else np.nan

            rows.append(
                {
                    "model1":  model_a,
                    "prompt1": gemma_prompt,
                    "model2":  model_b,
                    "prompt2": gpt_prompt,
                    "metric":  metric,
                    "n1":      n_a,
                    "n2":      n_b,
                    "mean1":   mean_a,
                    "mean2":   mean_b,
                    "t_stat":  t_stat,
                    "p_value": p_val,
                    "cohens_d": cohens_d,
                }
            )

    return pd.DataFrame(rows)

pairwise_prompt_tests = compare_prompt_pairs(
    df,
    metrics=metric_cols,
    prompt_pairs=prompt_pairs,
    model_a="Gemma",
    model_b="GPT-3.5",
)

print("\nMatched prompt-pair t-tests (Gemma vs GPT-3.5):")
if not pairwise_prompt_tests.empty:
    print(pairwise_prompt_tests.sort_values(["metric", "prompt1", "prompt2"]))
else:
    print("No matched prompt-pair tests could be computed (check prompt labels).")

###########################################
# 8. Save outputs (optional)
###########################################

desc.to_csv("desc_by_model_prompt.csv", index=False)
global_tests.to_csv("global_t_tests_gemma_vs_gpt.csv", index=False)
per_prompt_tests.to_csv("per_prompt_t_tests_gemma_vs_gpt_same_prompt.csv", index=False)
pairwise_prompt_tests.to_csv("pairwise_prompt_t_tests_gemma_vs_gpt.csv", index=False)


Gemma columns: ['Prompting_strategy', 'Question_Type', 'Question', 'Correct_Answer', 'Choice_1', 'Choice_2', 'Choice_3', 'Word_Difficulty', 'Task_Difficulty', 'Text', 'Overall_Score', 'Grammar_Score', 'Complexity_Score', 'Readability_Score', 'Fluency_Score', 'Error_Count', 'Errors', 'Unnamed: 17']
GPT columns:   ['Row', 'Prompting_strategy', 'Question_Type', 'Question', 'Correct_Answer', 'Word_Difficulty', 'Task_Difficulty', 'Text', 'Overall_Score', 'Grammar_Score', 'Complexity_Score', 'Readability_Score', 'Fluency_Score', 'Error_Count', 'Errors']

Combined df head:
       prompt  Question_Type  Overall_Score  Grammar_Score  Complexity_Score  \
0  CoT+Seq_Rl            1.0         73.589          100.0              80.0   
1  CoT+Seq_Rl            1.0         67.949          100.0              80.0   
2  CoT+Seq_Rl            1.0         65.129          100.0              80.0   
3  CoT+Seq_Rl            1.0         70.769          100.0              80.0   
4  CoT+Seq_Rl            1.

In [48]:
import pandas as pd
import numpy as np
from scipy import stats

#############################
# 1. Load and harmonize data
#############################

# Files and sheet names you specified
gemma_raw = pd.read_excel("Gemma_Resultss.xlsx", sheet_name="Gemma Results")
gpt_raw   = pd.read_excel("Results_GPTa.xlsx",   sheet_name="Results")

print("Gemma columns:", gemma_raw.columns.tolist())
print("GPT columns:  ", gpt_raw.columns.tolist())

###############################################
# 2. Select & rename relevant columns per model
###############################################

# Columns we care about from each file
common_cols_gemma = [
    "Prompting_strategy",
    "Question_Type",
    "Overall_Score",
    "Grammar_Score",
    "Complexity_Score",
    "Readability_Score",
    "Fluency_Score",
]

common_cols_gpt = [
    "Prompting_strategy",
    "Question_Type",
    "Overall_Score",
    "Grammar_Score",
    "Complexity_Score",
    "Readability_Score",
    "Fluency_Score",
]

# Subset safely (will raise if a column is missing → good sanity check)
gemma = gemma_raw[common_cols_gemma].copy()
gpt   = gpt_raw[common_cols_gpt].copy()

# Add model labels
gemma["model"] = "Gemma"
gpt["model"]   = "GPT-3.5"

# Normalize prompt column name
gemma.rename(columns={"Prompting_strategy": "prompt"}, inplace=True)
gpt.rename(columns={"Prompting_strategy": "prompt"}, inplace=True)

# Create lowercase metric aliases (easier to work with)
rename_metrics = {
    "Overall_Score":     "overall",
    "Grammar_Score":     "grammar",
    "Complexity_Score":  "complexity",
    "Readability_Score": "readability",
    "Fluency_Score":     "fluency",
}

for old, new in rename_metrics.items():
    gemma[new] = gemma[old]
    gpt[new]   = gpt[old]

metric_cols = list(rename_metrics.values())

#########################################
# 3. Combine into a single long dataframe
#########################################

df = pd.concat([gemma, gpt], ignore_index=True)

# Clean numeric types
for c in metric_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

print("\nCombined df head:")
print(df.head())

##############################################
# 4. Descriptive stats by model × prompt type
##############################################

group_cols = ["model", "prompt"]
desc = (
    df.groupby(group_cols)[metric_cols]
      .agg(["mean", "std", "count"])
)

# Flatten MultiIndex columns
desc.columns = [f"{m}_{stat}" for m, stat in desc.columns]
desc = desc.reset_index()

print("\nDescriptive statistics by model × prompting strategy:")
print(desc)

############################################
# 5. Global independent t-tests (Gemma vs GPT)
############################################

def global_t_tests(df, metrics, model_a="Gemma", model_b="GPT-3.5"):
    """Global (across all prompts) Welch t-tests for each metric."""
    rows = []
    for metric in metrics:
        a_vals = df.loc[df["model"] == model_a, metric].dropna().values
        b_vals = df.loc[df["model"] == model_b, metric].dropna().values

        if len(a_vals) < 2 or len(b_vals) < 2:
            continue

        # Welch t-test
        t_stat, p_val = stats.ttest_ind(a_vals, b_vals, equal_var=False)

        mean_a, mean_b = a_vals.mean(), b_vals.mean()
        sd_a, sd_b = a_vals.std(ddof=1), b_vals.std(ddof=1)
        n_a, n_b = len(a_vals), len(b_vals)

        pooled_sd = np.sqrt(((n_a - 1)*sd_a**2 + (n_b - 1)*sd_b**2) / (n_a + n_b - 2))
        cohens_d = (mean_a - mean_b) / pooled_sd if pooled_sd > 0 else np.nan

        rows.append(
            {
                "metric": metric,
                "n_gemma": n_a,
                "n_gpt": n_b,
                "mean_gemma": mean_a,
                "mean_gpt": mean_b,
                "t_stat": t_stat,
                "p_value": p_val,
                "cohens_d": cohens_d,
            }
        )

    return pd.DataFrame(rows)

global_tests = global_t_tests(df, metrics=metric_cols)
print("\nGlobal t-tests (Gemma vs GPT-3.5) across ALL prompts:")
print(global_tests)

##############################################################
# 6. Per-prompt independent t-tests (same prompt name in both)
##############################################################

def independent_t_tests_by_prompt(df, metric, model_a="Gemma", model_b="GPT-3.5"):
    """
    Welch t-test for each *prompt name* separately, for one metric,
    assuming both models share the same prompt label.
    """
    rows = []
    for prompt in df["prompt"].unique():
        sub = df[df["prompt"] == prompt]

        a_vals = sub.loc[sub["model"] == model_a, metric].dropna().values
        b_vals = sub.loc[sub["model"] == model_b, metric].dropna().values

        # Need at least 2 per group for a meaningful test
        if len(a_vals) < 2 or len(b_vals) < 2:
            continue

        t_stat, p_val = stats.ttest_ind(a_vals, b_vals, equal_var=False)

        mean_a, mean_b = a_vals.mean(), b_vals.mean()
        sd_a, sd_b = a_vals.std(ddof=1), b_vals.std(ddof=1)
        n_a, n_b = len(a_vals), len(b_vals)

        pooled_sd = np.sqrt(((n_a - 1)*sd_a**2 + (n_b - 1)*sd_b**2) / (n_a + n_b - 2))
        cohens_d = (mean_a - mean_b) / pooled_sd if pooled_sd > 0 else np.nan

        rows.append(
            {
                "prompt": prompt,
                "metric": metric,
                "n_gemma": n_a,
                "n_gpt": n_b,
                "mean_gemma": mean_a,
                "mean_gpt": mean_b,
                "t_stat": t_stat,
                "p_value": p_val,
                "cohens_d": cohens_d,
            }
        )

    return pd.DataFrame(rows)

per_prompt_results_list = []
for m in metric_cols:
    res_m = independent_t_tests_by_prompt(df, metric=m)
    print(f"Metric {m}: {len(res_m)} per-prompt rows")
    if not res_m.empty:
        per_prompt_results_list.append(res_m)

if per_prompt_results_list:
    per_prompt_tests = pd.concat(per_prompt_results_list, ignore_index=True)
    print("\nPer-prompt t-tests (Gemma vs GPT-3.5), same prompt name:")
    sort_cols = [c for c in ["metric", "prompt"] if c in per_prompt_tests.columns]
    if sort_cols:
        print(per_prompt_tests.sort_values(sort_cols))
    else:
        print(per_prompt_tests)
else:
    per_prompt_tests = pd.DataFrame()
    print("\nNo per-prompt t-tests could be computed "
          "(likely < 2 items per model per prompt for each metric).")

###################################################################
# 7. Cross-prompt specs: ALL pairs you want to check
###################################################################

cross_prompt_specs = [
    # Gemma CoT vs all GPT prompts
    {"model1": "Gemma",   "prompt1": "CoT",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought"},

    {"model1": "Gemma",   "prompt1": "CoT",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_role_chain"},

    {"model1": "Gemma",   "prompt1": "CoT",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_sequential"},

    {"model1": "Gemma",   "prompt1": "CoT",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_sequential_rl"},

    {"model1": "Gemma",   "prompt1": "CoT",
     "model2": "GPT-3.5", "prompt2": "few_shot"},

    {"model1": "Gemma",   "prompt1": "CoT",
     "model2": "GPT-3.5", "prompt2": "zero_shot"},

    # Gemma CoT+RC vs all GPT prompts
    {"model1": "Gemma",   "prompt1": "CoT+RC",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought"},

    {"model1": "Gemma",   "prompt1": "CoT+RC",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_role_chain"},

    {"model1": "Gemma",   "prompt1": "CoT+RC",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_sequential"},

    {"model1": "Gemma",   "prompt1": "CoT+RC",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_sequential_rl"},

    {"model1": "Gemma",   "prompt1": "CoT+RC",
     "model2": "GPT-3.5", "prompt2": "few_shot"},

    {"model1": "Gemma",   "prompt1": "CoT+RC",
     "model2": "GPT-3.5", "prompt2": "zero_shot"},

    # Gemma CoT+Seq vs all GPT prompts
    {"model1": "Gemma",   "prompt1": "CoT+Seq",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought"},

    {"model1": "Gemma",   "prompt1": "CoT+Seq",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_role_chain"},

    {"model1": "Gemma",   "prompt1": "CoT+Seq",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_sequential"},

    {"model1": "Gemma",   "prompt1": "CoT+Seq",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_sequential_rl"},

    {"model1": "Gemma",   "prompt1": "CoT+Seq",
     "model2": "GPT-3.5", "prompt2": "few_shot"},

    {"model1": "Gemma",   "prompt1": "CoT+Seq",
     "model2": "GPT-3.5", "prompt2": "zero_shot"},

    # Gemma CoT+Seq_Rl vs all GPT prompts
    {"model1": "Gemma",   "prompt1": "CoT+Seq_Rl",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought"},

    {"model1": "Gemma",   "prompt1": "CoT+Seq_Rl",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_role_chain"},

    {"model1": "Gemma",   "prompt1": "CoT+Seq_Rl",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_sequential"},

    {"model1": "Gemma",   "prompt1": "CoT+Seq_Rl",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_sequential_rl"},

    {"model1": "Gemma",   "prompt1": "CoT+Seq_Rl",
     "model2": "GPT-3.5", "prompt2": "few_shot"},

    {"model1": "Gemma",   "prompt1": "CoT+Seq_Rl",
     "model2": "GPT-3.5", "prompt2": "zero_shot"},

    # Gemma Few_Shot vs all GPT prompts
    {"model1": "Gemma",   "prompt1": "Few_Shot",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought"},

    {"model1": "Gemma",   "prompt1": "Few_Shot",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_role_chain"},

    {"model1": "Gemma",   "prompt1": "Few_Shot",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_sequential"},

    {"model1": "Gemma",   "prompt1": "Few_Shot",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_sequential_rl"},

    {"model1": "Gemma",   "prompt1": "Few_Shot",
     "model2": "GPT-3.5", "prompt2": "few_shot"},

    {"model1": "Gemma",   "prompt1": "Few_Shot",
     "model2": "GPT-3.5", "prompt2": "zero_shot"},

    # Gemma Zero_Shot vs all GPT prompts
    {"model1": "Gemma",   "prompt1": "Zero_Shot",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought"},

    {"model1": "Gemma",   "prompt1": "Zero_Shot",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_role_chain"},

    {"model1": "Gemma",   "prompt1": "Zero_Shot",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_sequential"},

    {"model1": "Gemma",   "prompt1": "Zero_Shot",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_sequential_rl"},

    {"model1": "Gemma",   "prompt1": "Zero_Shot",
     "model2": "GPT-3.5", "prompt2": "few_shot"},

    {"model1": "Gemma",   "prompt1": "Zero_Shot",
     "model2": "GPT-3.5", "prompt2": "zero_shot"},
]

###################################################################
# 8. Run cross-prompt t-tests for ALL metrics over these pairs
###################################################################

def run_cross_prompt_tests(df, metrics, specs):
    """
    For each spec dict (model1, prompt1, model2, prompt2) and each metric,
    run a Welch t-test and compute Cohen's d.
    """
    rows = []
    for spec in specs:
        m1 = spec["model1"]
        p1 = spec["prompt1"]
        m2 = spec["model2"]
        p2 = spec["prompt2"]

        for metric in metrics:
            v1 = df.loc[(df["model"] == m1) & (df["prompt"] == p1), metric].dropna().values
            v2 = df.loc[(df["model"] == m2) & (df["prompt"] == p2), metric].dropna().values

            if len(v1) < 2 or len(v2) < 2:
                # Not enough data for a meaningful test
                continue

            t_stat, p_val = stats.ttest_ind(v1, v2, equal_var=False)

            mean1, mean2 = v1.mean(), v2.mean()
            sd1, sd2 = v1.std(ddof=1), v2.std(ddof=1)
            n1, n2 = len(v1), len(v2)

            pooled_sd = np.sqrt(((n1 - 1)*sd1**2 + (n2 - 1)*sd2**2) / (n1 + n2 - 2))
            cohens_d = (mean1 - mean2) / pooled_sd if pooled_sd > 0 else np.nan

            rows.append(
                {
                    "model1":  m1,
                    "prompt1": p1,
                    "model2":  m2,
                    "prompt2": p2,
                    "metric":  metric,
                    "n1":      n1,
                    "n2":      n2,
                    "mean1":   mean1,
                    "mean2":   mean2,
                    "t_stat":  t_stat,
                    "p_value": p_val,
                    "cohens_d": cohens_d,
                }
            )

    return pd.DataFrame(rows)

cross_prompt_tests = run_cross_prompt_tests(df, metrics=metric_cols, specs=cross_prompt_specs)

print("\nCross-prompt t-tests (Gemma vs GPT-3.5) for all specified pairs:")
if not cross_prompt_tests.empty:
    print(cross_prompt_tests.sort_values(["prompt1", "prompt2", "metric"]))
else:
    print("No cross-prompt tests could be computed (check labels / sample sizes).")

###########################################
# 9. Save outputs (optional)
###########################################

desc.to_csv("desc_by_model_prompt.csv", index=False)
global_tests.to_csv("global_t_tests_gemma_vs_gpt.csv", index=False)
per_prompt_tests.to_csv("per_prompt_t_tests_gemma_vs_gpt_same_prompt.csv", index=False)
cross_prompt_tests.to_csv("cross_prompt_t_tests_gemma_vs_gpt.csv", index=False)


Gemma columns: ['Prompting_strategy', 'Question_Type', 'Question', 'Correct_Answer', 'Choice_1', 'Choice_2', 'Choice_3', 'Word_Difficulty', 'Task_Difficulty', 'Text', 'Overall_Score', 'Grammar_Score', 'Complexity_Score', 'Readability_Score', 'Fluency_Score', 'Error_Count', 'Errors', 'Unnamed: 17']
GPT columns:   ['Row', 'Prompting_strategy', 'Question_Type', 'Question', 'Correct_Answer', 'Word_Difficulty', 'Task_Difficulty', 'Text', 'Overall_Score', 'Grammar_Score', 'Complexity_Score', 'Readability_Score', 'Fluency_Score', 'Error_Count', 'Errors']

Combined df head:
       prompt  Question_Type  Overall_Score  Grammar_Score  Complexity_Score  \
0  CoT+Seq_Rl            1.0         73.589          100.0              80.0   
1  CoT+Seq_Rl            1.0         67.949          100.0              80.0   
2  CoT+Seq_Rl            1.0         65.129          100.0              80.0   
3  CoT+Seq_Rl            1.0         70.769          100.0              80.0   
4  CoT+Seq_Rl            1.

In [52]:
import pandas as pd
import numpy as np

# ----------------------------------------------------
# 1. Load cross-prompt t-test results from CSV
# ----------------------------------------------------
# Make sure the file name matches what you saved previously
cross = pd.read_csv("cross_prompt_t_tests_gemma_vs_gpt.csv")

print("Columns in cross-prompt file:", cross.columns.tolist())

# ----------------------------------------------------
# 2. Filter to 'overall' metric only (if desired)
# ----------------------------------------------------
overall = cross[cross["metric"] == "overall"].copy()

# Optional: sort rows for a nice layout
overall = overall.sort_values(
    by=["model1", "prompt1", "model2", "prompt2"]
).reset_index(drop=True)

# ----------------------------------------------------
# 3. Select columns for the paper table
# ----------------------------------------------------
cols_for_table = [
    "model1", "prompt1",
    "model2", "prompt2",
    "n1", "n2",
    "mean1", "mean2",
    "p_value",
    "cohens_d",
]

table_df = overall[cols_for_table].copy()

# ----------------------------------------------------
# 4. Nicely format numeric columns
# ----------------------------------------------------
# Adjust decimal places as you prefer
table_df["mean1"]   = table_df["mean1"].map(lambda x: f"{x:.2f}")
table_df["mean2"]   = table_df["mean2"].map(lambda x: f"{x:.2f}")
table_df["p_value"] = table_df["p_value"].map(lambda x: f"{x:.3f}")
table_df["cohens_d"] = table_df["cohens_d"].map(lambda x: f"{x:.3f}")

# Optional: highlight significant p-values in bold (p < 0.05)
def format_p(p_str):
    try:
        val = float(p_str)
        if val < 0.05:
            return f"\\textbf{{{p_str}}}"
        else:
            return p_str
    except ValueError:
        return p_str

table_df["p_value"] = table_df["p_value"].apply(format_p)

# ----------------------------------------------------
# 5. Rename columns for LaTeX header
# ----------------------------------------------------
table_df = table_df.rename(columns={
    "model1":   "Model 1",
    "prompt1":  "Prompt 1",
    "model2":   "Model 2",
    "prompt2":  "Prompt 2",
    "n1":       "$n_1$",
    "n2":       "$n_2$",
    "mean1":    "Mean 1",
    "mean2":    "Mean 2",
    "p_value":  "$p$-value",
    "cohens_d": "Cohen's $d$",
})

# ----------------------------------------------------
# 6. Export as LaTeX tabular (without table environment)
# ----------------------------------------------------
latex_tabular = table_df.to_latex(
    index=False,
    escape=False,     # allow LaTeX commands like \textbf in cells
    column_format="llllrrllll"  # adjust alignment if desired
)

print("\nLaTeX tabular code:\n")
print(latex_tabular)

# ----------------------------------------------------
# 7. (Optional) Write to a .tex file
# ----------------------------------------------------
with open("cross_prompt_overall_ttests_table.tex", "w") as f:
    f.write(latex_tabular)


Columns in cross-prompt file: ['model1', 'prompt1', 'model2', 'prompt2', 'metric', 'n1', 'n2', 'mean1', 'mean2', 't_stat', 'p_value', 'cohens_d']

LaTeX tabular code:

\begin{tabular}{llllrrllll}
\toprule
Model 1 & Prompt 1 & Model 2 & Prompt 2 & $n_1$ & $n_2$ & Mean 1 & Mean 2 & $p$-value & Cohen's $d$ \\
\midrule
Gemma & CoT & GPT-3.5 & chain_of_thought & 391 & 585 & 77.43 & 78.58 & 0.250 & -0.079 \\
Gemma & CoT & GPT-3.5 & chain_of_thought_plus_role_chain & 391 & 585 & 77.43 & 78.31 & 0.400 & -0.056 \\
Gemma & CoT & GPT-3.5 & chain_of_thought_plus_sequential & 391 & 584 & 77.43 & 78.54 & 0.292 & -0.070 \\
Gemma & CoT & GPT-3.5 & chain_of_thought_plus_sequential_rl & 391 & 585 & 77.43 & 78.13 & 0.488 & -0.047 \\
Gemma & CoT & GPT-3.5 & few_shot & 391 & 585 & 77.43 & 81.95 & \textbf{0.000} & -0.318 \\
Gemma & CoT & GPT-3.5 & zero_shot & 391 & 585 & 77.43 & 79.42 & \textbf{0.039} & -0.146 \\
Gemma & CoT+RC & GPT-3.5 & chain_of_thought & 329 & 585 & 77.12 & 78.58 & 0.125 & -0.108 \\
Gem

In [56]:
import pandas as pd
import numpy as np
from scipy import stats

#############################
# 1. Load and harmonize data
#############################

# Files and sheet names you specified
gemma_raw = pd.read_excel("Gemma_Resultss.xlsx", sheet_name="Gemma Results")
gpt_raw   = pd.read_excel("Results_GPTa.xlsx",   sheet_name="Results")

print("Gemma columns:", gemma_raw.columns.tolist())
print("GPT columns:  ", gpt_raw.columns.tolist())

###############################################
# 2. Select & rename relevant columns per model
###############################################

# Columns we care about from each file
common_cols_gemma = [
    "Prompting_strategy",
    "Question_Type",
    "Overall_Score",
    "Grammar_Score",
    "Complexity_Score",
    "Readability_Score",
    "Fluency_Score",
]

common_cols_gpt = [
    "Prompting_strategy",
    "Question_Type",
    "Overall_Score",
    "Grammar_Score",
    "Complexity_Score",
    "Readability_Score",
    "Fluency_Score",
]

# Subset safely (will raise if a column is missing → good sanity check)
gemma = gemma_raw[common_cols_gemma].copy()
gpt   = gpt_raw[common_cols_gpt].copy()

# Add model labels
gemma["model"] = "Gemma"
gpt["model"]   = "GPT-3.5"

# Normalize prompt column name
gemma.rename(columns={"Prompting_strategy": "prompt"}, inplace=True)
gpt.rename(columns={"Prompting_strategy": "prompt"}, inplace=True)

# Create lowercase metric aliases (easier to work with)
rename_metrics = {
    "Overall_Score":     "overall",
    "Grammar_Score":     "grammar",
    "Complexity_Score":  "complexity",
    "Readability_Score": "readability",
    "Fluency_Score":     "fluency",
}

for old, new in rename_metrics.items():
    gemma[new] = gemma[old]
    gpt[new]   = gpt[old]

metric_cols = list(rename_metrics.values())

#########################################
# 3. Combine into a single long dataframe
#########################################

df = pd.concat([gemma, gpt], ignore_index=True)

# Clean numeric types
for c in metric_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

print("\nCombined df head:")
print(df.head())

##############################################
# 4. Descriptive stats by model × prompt type
##############################################

group_cols = ["model", "prompt"]
desc = (
    df.groupby(group_cols)[metric_cols]
      .agg(["mean", "std", "count"])
)

# Flatten MultiIndex columns
desc.columns = [f"{m}_{stat}" for m, stat in desc.columns]
desc = desc.reset_index()

print("\nDescriptive statistics by model × prompting strategy:")
print(desc)

############################################
# 5. Global independent t-tests (Gemma vs GPT)
############################################

def global_t_tests(df, metrics, model_a="Gemma", model_b="GPT-3.5"):
    """Global (across all prompts) Welch t-tests for each metric."""
    rows = []
    for metric in metrics:
        a_vals = df.loc[df["model"] == model_a, metric].dropna().values
        b_vals = df.loc[df["model"] == model_b, metric].dropna().values

        if len(a_vals) < 2 or len(b_vals) < 2:
            continue

        # Welch t-test
        t_stat, p_val = stats.ttest_ind(a_vals, b_vals, equal_var=False)

        mean_a, mean_b = a_vals.mean(), b_vals.mean()
        sd_a, sd_b = a_vals.std(ddof=1), b_vals.std(ddof=1)
        n_a, n_b = len(a_vals), len(b_vals)

        pooled_sd = np.sqrt(((n_a - 1)*sd_a**2 + (n_b - 1)*sd_b**2) / (n_a + n_b - 2))
        cohens_d = (mean_a - mean_b) / pooled_sd if pooled_sd > 0 else np.nan

        rows.append(
            {
                "metric":      metric,
                "n_gemma":     n_a,
                "n_gpt":       n_b,
                "mean_gemma":  mean_a,
                "mean_gpt":    mean_b,
                "t_stat":      t_stat,
                "p_value":     p_val,
                "cohens_d":    cohens_d,
            }
        )

    return pd.DataFrame(rows)

global_tests = global_t_tests(df, metrics=metric_cols)
print("\nGlobal t-tests (Gemma vs GPT-3.5) across ALL prompts:")
print(global_tests)

##############################################################
# 6. Per-prompt independent t-tests (same prompt name in both)
##############################################################

def independent_t_tests_by_prompt(df, metric, model_a="Gemma", model_b="GPT-3.5"):
    """
    Welch t-test for each *prompt name* separately, for one metric,
    assuming both models share the same prompt label.
    """
    rows = []
    for prompt in df["prompt"].unique():
        sub = df[df["prompt"] == prompt]

        a_vals = sub.loc[sub["model"] == model_a, metric].dropna().values
        b_vals = sub.loc[sub["model"] == model_b, metric].dropna().values

        # Need at least 2 per group for a meaningful test
        if len(a_vals) < 2 or len(b_vals) < 2:
            continue

        t_stat, p_val = stats.ttest_ind(a_vals, b_vals, equal_var=False)

        mean_a, mean_b = a_vals.mean(), b_vals.mean()
        sd_a, sd_b = a_vals.std(ddof=1), b_vals.std(ddof=1)
        n_a, n_b = len(a_vals), len(b_vals)

        pooled_sd = np.sqrt(((n_a - 1)*sd_a**2 + (n_b - 1)*sd_b**2) / (n_a + n_b - 2))
        cohens_d = (mean_a - mean_b) / pooled_sd if pooled_sd > 0 else np.nan

        rows.append(
            {
                "prompt":      prompt,
                "metric":      metric,
                "n_gemma":     n_a,
                "n_gpt":       n_b,
                "mean_gemma":  mean_a,
                "mean_gpt":    mean_b,
                "t_stat":      t_stat,
                "p_value":     p_val,
                "cohens_d":    cohens_d,
            }
        )

    return pd.DataFrame(rows)

per_prompt_results_list = []
for m in metric_cols:
    res_m = independent_t_tests_by_prompt(df, metric=m)
    print(f"Metric {m}: {len(res_m)} per-prompt rows")
    if not res_m.empty:
        per_prompt_results_list.append(res_m)

if per_prompt_results_list:
    per_prompt_tests = pd.concat(per_prompt_results_list, ignore_index=True)
    print("\nPer-prompt t-tests (Gemma vs GPT-3.5), same prompt name:")
    sort_cols = [c for c in ["metric", "prompt"] if c in per_prompt_tests.columns]
    if sort_cols:
        print(per_prompt_tests.sort_values(sort_cols))
    else:
        print(per_prompt_tests)
else:
    per_prompt_tests = pd.DataFrame()
    print("\nNo per-prompt t-tests could be computed "
          "(likely < 2 items per model per prompt for each metric).")

###################################################################
# 7. Cross-prompt specs: ALL pairs you want to check
###################################################################

cross_prompt_specs = [
    # Gemma CoT vs all GPT prompts
    {"model1": "Gemma",   "prompt1": "CoT",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought"},

    {"model1": "Gemma",   "prompt1": "CoT",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_role_chain"},

    {"model1": "Gemma",   "prompt1": "CoT",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_sequential"},

    {"model1": "Gemma",   "prompt1": "CoT",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_sequential_rl"},

    {"model1": "Gemma",   "prompt1": "CoT",
     "model2": "GPT-3.5", "prompt2": "few_shot"},

    {"model1": "Gemma",   "prompt1": "CoT",
     "model2": "GPT-3.5", "prompt2": "zero_shot"},

    # Gemma CoT+RC vs all GPT prompts
    {"model1": "Gemma",   "prompt1": "CoT+RC",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought"},

    {"model1": "Gemma",   "prompt1": "CoT+RC",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_role_chain"},

    {"model1": "Gemma",   "prompt1": "CoT+RC",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_sequential"},

    {"model1": "Gemma",   "prompt1": "CoT+RC",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_sequential_rl"},

    {"model1": "Gemma",   "prompt1": "CoT+RC",
     "model2": "GPT-3.5", "prompt2": "few_shot"},

    {"model1": "Gemma",   "prompt1": "CoT+RC",
     "model2": "GPT-3.5", "prompt2": "zero_shot"},

    # Gemma CoT+Seq vs all GPT prompts
    {"model1": "Gemma",   "prompt1": "CoT+Seq",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought"},

    {"model1": "Gemma",   "prompt1": "CoT+Seq",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_role_chain"},

    {"model1": "Gemma",   "prompt1": "CoT+Seq",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_sequential"},

    {"model1": "Gemma",   "prompt1": "CoT+Seq",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_sequential_rl"},

    {"model1": "Gemma",   "prompt1": "CoT+Seq",
     "model2": "GPT-3.5", "prompt2": "few_shot"},

    {"model1": "Gemma",   "prompt1": "CoT+Seq",
     "model2": "GPT-3.5", "prompt2": "zero_shot"},

    # Gemma CoT+Seq_Rl vs all GPT prompts
    {"model1": "Gemma",   "prompt1": "CoT+Seq_Rl",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought"},

    {"model1": "Gemma",   "prompt1": "CoT+Seq_Rl",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_role_chain"},

    {"model1": "Gemma",   "prompt1": "CoT+Seq_Rl",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_sequential"},

    {"model1": "Gemma",   "prompt1": "CoT+Seq_Rl",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_sequential_rl"},

    {"model1": "Gemma",   "prompt1": "CoT+Seq_Rl",
     "model2": "GPT-3.5", "prompt2": "few_shot"},

    {"model1": "Gemma",   "prompt1": "CoT+Seq_Rl",
     "model2": "GPT-3.5", "prompt2": "zero_shot"},

    # Gemma Few_Shot vs all GPT prompts
    {"model1": "Gemma",   "prompt1": "Few_Shot",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought"},

    {"model1": "Gemma",   "prompt1": "Few_Shot",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_role_chain"},

    {"model1": "Gemma",   "prompt1": "Few_Shot",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_sequential"},

    {"model1": "Gemma",   "prompt1": "Few_Shot",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_sequential_rl"},

    {"model1": "Gemma",   "prompt1": "Few_Shot",
     "model2": "GPT-3.5", "prompt2": "few_shot"},

    {"model1": "Gemma",   "prompt1": "Few_Shot",
     "model2": "GPT-3.5", "prompt2": "zero_shot"},

    # Gemma Zero_Shot vs all GPT prompts
    {"model1": "Gemma",   "prompt1": "Zero_Shot",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought"},

    {"model1": "Gemma",   "prompt1": "Zero_Shot",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_role_chain"},

    {"model1": "Gemma",   "prompt1": "Zero_Shot",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_sequential"},

    {"model1": "Gemma",   "prompt1": "Zero_Shot",
     "model2": "GPT-3.5", "prompt2": "chain_of_thought_plus_sequential_rl"},

    {"model1": "Gemma",   "prompt1": "Zero_Shot",
     "model2": "GPT-3.5", "prompt2": "few_shot"},

    {"model1": "Gemma",   "prompt1": "Zero_Shot",
     "model2": "GPT-3.5", "prompt2": "zero_shot"},
]

###################################################################
# 8. Run cross-prompt t-tests for ALL metrics over these pairs
###################################################################

def run_cross_prompt_tests(df, metrics, specs):
    """
    For each spec dict (model1, prompt1, model2, prompt2) and each metric,
    run a Welch t-test and compute Cohen's d, including sd1 and sd2.
    """
    rows = []
    for spec in specs:
        m1 = spec["model1"]
        p1 = spec["prompt1"]
        m2 = spec["model2"]
        p2 = spec["prompt2"]

        for metric in metrics:
            v1 = df.loc[(df["model"] == m1) & (df["prompt"] == p1), metric].dropna().values
            v2 = df.loc[(df["model"] == m2) & (df["prompt"] == p2), metric].dropna().values

            if len(v1) < 2 or len(v2) < 2:
                # Not enough data for a meaningful test
                continue

            t_stat, p_val = stats.ttest_ind(v1, v2, equal_var=False)

            mean1, mean2 = v1.mean(), v2.mean()
            sd1, sd2 = v1.std(ddof=1), v2.std(ddof=1)
            n1, n2 = len(v1), len(v2)

            pooled_sd = np.sqrt(((n1 - 1)*sd1**2 + (n2 - 1)*sd2**2) / (n1 + n2 - 2))
            cohens_d = (mean1 - mean2) / pooled_sd if pooled_sd > 0 else np.nan

            rows.append(
                {
                    "model1":   m1,
                    "prompt1":  p1,
                    "model2":   m2,
                    "prompt2":  p2,
                    "metric":   metric,
                    "n1":       n1,
                    "n2":       n2,
                    "mean1":    mean1,
                    "sd1":      sd1,
                    "mean2":    mean2,
                    "sd2":      sd2,
                    "t_stat":   t_stat,
                    "p_value":  p_val,
                    "cohens_d": cohens_d,
                }
            )

    return pd.DataFrame(rows)

cross_prompt_tests = run_cross_prompt_tests(df, metrics=metric_cols, specs=cross_prompt_specs)

print("\nCross-prompt t-tests (Gemma vs GPT-3.5) for all specified pairs:")
if not cross_prompt_tests.empty:
    print(cross_prompt_tests.sort_values(["prompt1", "prompt2", "metric"]))
else:
    print("No cross-prompt tests could be computed (check labels / sample sizes).")

###########################################
# 9. Save outputs (optional)
###########################################

desc.to_csv("adesc_by_model_prompt.csv", index=False)
global_tests.to_csv("AGlobal_t_tests_gemma_vs_gpt.csv", index=False)
per_prompt_tests.to_csv("APer_prompt_t_tests_gemma_vs_gpt_same_prompt.csv", index=False)
cross_prompt_tests.to_csv("ACross_prompt_t_tests_gemma_vs_gpt.csv", index=False)


Gemma columns: ['Prompting_strategy', 'Question_Type', 'Question', 'Correct_Answer', 'Choice_1', 'Choice_2', 'Choice_3', 'Word_Difficulty', 'Task_Difficulty', 'Text', 'Overall_Score', 'Grammar_Score', 'Complexity_Score', 'Readability_Score', 'Fluency_Score', 'Error_Count', 'Errors', 'Unnamed: 17']
GPT columns:   ['Row', 'Prompting_strategy', 'Question_Type', 'Question', 'Correct_Answer', 'Word_Difficulty', 'Task_Difficulty', 'Text', 'Overall_Score', 'Grammar_Score', 'Complexity_Score', 'Readability_Score', 'Fluency_Score', 'Error_Count', 'Errors']

Combined df head:
       prompt  Question_Type  Overall_Score  Grammar_Score  Complexity_Score  \
0  CoT+Seq_Rl            1.0         73.589          100.0              80.0   
1  CoT+Seq_Rl            1.0         67.949          100.0              80.0   
2  CoT+Seq_Rl            1.0         65.129          100.0              80.0   
3  CoT+Seq_Rl            1.0         70.769          100.0              80.0   
4  CoT+Seq_Rl            1.

In [66]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

# ----------------------------
# Helpers: text normalization & Jaccard
# ----------------------------

def normalize_and_tokenize(text):
    """
    Lowercase, remove punctuation, and split into tokens.
    Returns a set of tokens for Jaccard similarity.
    """
    if pd.isna(text):
        return set()
    text = str(text).lower()
    # Keep only letters, digits, and spaces
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    if not text:
        return set()
    return set(text.split())

def jaccard_similarity(set_a, set_b):
    if not set_a and not set_b:
        return 0.0
    inter = len(set_a & set_b)
    union = len(set_a | set_b)
    if union == 0:
        return 0.0
    return inter / union

def compute_within_file_max_jaccard(df, text_cols, id_col_name=None):
    """
    For each row:
      - Concatenate text_cols into a single string
      - Tokenize
      - Compute max Jaccard vs all other rows in the same file
    """
    df = df.copy()

    # Combine selected text columns
    df["combined_text"] = (
        df[text_cols]
        .fillna("")
        .astype(str)
        .agg(" ".join, axis=1)
        .str.strip()
    )

    # Tokenize all rows once
    token_sets = [normalize_and_tokenize(t) for t in df["combined_text"]]

    n = len(df)
    max_jaccard = np.zeros(n, dtype=float)

    for i in tqdm(range(n), desc="Computing max Jaccard within file"):
        s_i = token_sets[i]
        best = 0.0
        for j in range(n):
            if i == j:
                continue
            s_j = token_sets[j]
            sim = jaccard_similarity(s_i, s_j)
            if sim > best:
                best = sim
        max_jaccard[i] = best

    df["max_jaccard_within"] = max_jaccard

    if id_col_name is not None and id_col_name not in df.columns:
        df[id_col_name] = np.arange(1, n + 1)

    return df

# ----------------------------
# Robust Excel reader (handles extra header row)
# ----------------------------

def read_excel_with_required_cols(path, required_cols):
    """
    Try reading Excel with default header=0.
    If required columns are missing, try header=1 (skip first row),
    which often fixes 'Unnamed: 0' header issues.
    """
    print(f"\nReading: {path} with header=0")
    df0 = pd.read_excel(path, header=0)
    df0.columns = df0.columns.map(lambda x: str(x).strip())
    print("Columns (header=0):", list(df0.columns))

    if required_cols.issubset(set(df0.columns)):
        print("✅ Found required columns with header=0")
        return df0

    print("Required columns not found with header=0. Trying header=1...")
    df1 = pd.read_excel(path, header=1)
    df1.columns = df1.columns.map(lambda x: str(x).strip())
    print("Columns (header=1):", list(df1.columns))

    if required_cols.issubset(set(df1.columns)):
        print("✅ Found required columns with header=1")
        return df1

    raise KeyError(
        f"Could not find required columns {required_cols} in {path}.\n"
        f"header=0 columns: {list(df0.columns)}\n"
        f"header=1 columns: {list(df1.columns)}"
    )

# ----------------------------
# GPT: Question + Correct_Answer
# ----------------------------

# You can change this to your actual filename if different
gpt_path = "Results_GPTa.xlsx"

required_gpt_cols = {"Question", "Correct_Answer"}

df_gpt = read_excel_with_required_cols(gpt_path, required_gpt_cols)

print("\nFinal GPT columns:", list(df_gpt.columns))

gpt_text_cols = ["Question", "Correct_Answer"]

print("\nComputing Jaccard for GPT (Question + Correct_Answer)...")
df_gpt_jacc = compute_within_file_max_jaccard(
    df_gpt,
    text_cols=gpt_text_cols,
    id_col_name="Item_ID"
)

gpt_out_path = "results_GPT_with_jaccard.xlsx"
df_gpt_jacc.to_excel(gpt_out_path, index=False)
print(f"\nSaved GPT Jaccard results to: {gpt_out_path}")

print("\nGPT max Jaccard summary:")
print(df_gpt_jacc["max_jaccard_within"].describe())

# ----------------------------
# Gemma: Question + Choice_1 + Choice_2 + Choice_3
# ----------------------------

gemma_path = "Gemma_Resultss.xlsx"  # adjust if needed


required_gemma_cols = {"Question", "Correct_Answer"}

df_gemma = read_excel_with_required_cols(gemma_path, required_gemma_cols)

print("\nFinal Gemma columns:", list(df_gemma.columns))

gemma_text_cols = ["Question", "Choice_1", "Choice_2", "Choice_3"]

print("\nComputing Jaccard for Gemma (Question + 3 choices)...")
df_gemma_jacc = compute_within_file_max_jaccard(
    df_gemma,
    text_cols=gemma_text_cols,
    id_col_name="Item_ID"
)

gemma_out_path = "Gemma_Results_with_jaccard.xlsx"
df_gemma_jacc.to_excel(gemma_out_path, index=False)
print(f"\nSaved Gemma Jaccard results to: {gemma_out_path}")

print("\nGemma max Jaccard summary:")
print(df_gemma_jacc["max_jaccard_within"].describe())



Reading: Results_GPTa.xlsx with header=0
Columns (header=0): ['Row', 'Prompting_strategy', 'Question_Type', 'Question', 'Correct_Answer', 'Word_Difficulty', 'Task_Difficulty', 'Text', 'Overall_Score', 'Grammar_Score', 'Complexity_Score', 'Readability_Score', 'Fluency_Score', 'Error_Count', 'Errors']
✅ Found required columns with header=0

Final GPT columns: ['Row', 'Prompting_strategy', 'Question_Type', 'Question', 'Correct_Answer', 'Word_Difficulty', 'Task_Difficulty', 'Text', 'Overall_Score', 'Grammar_Score', 'Complexity_Score', 'Readability_Score', 'Fluency_Score', 'Error_Count', 'Errors']

Computing Jaccard for GPT (Question + Correct_Answer)...


Computing max Jaccard within file: 100%|███| 3509/3509 [00:09<00:00, 378.44it/s]



Saved GPT Jaccard results to: results_GPT_with_jaccard.xlsx

GPT max Jaccard summary:
count    3509.000000
mean        0.838467
std         0.178802
min         0.256410
25%         0.714286
50%         0.882353
75%         1.000000
max         1.000000
Name: max_jaccard_within, dtype: float64

Reading: Gemma_Resultss.xlsx with header=0
Columns (header=0): ['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4']
Required columns not found with header=0. Trying header=1...
Columns (header=1): ['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4']


KeyError: "Could not find required columns {'Choice_3', 'Choice_2', 'Choice_1', 'Question'} in Gemma_Resultss.xlsx.\nheader=0 columns: ['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4']\nheader=1 columns: ['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4']"

In [76]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

# ----------------------------
# Helpers: text normalization & Jaccard
# ----------------------------

def normalize_and_tokenize(text):
    """
    Lowercase, remove punctuation, and split into tokens.
    Returns a set of tokens for Jaccard similarity.
    """
    if pd.isna(text):
        return set()
    text = str(text).lower()
    # Keep only letters, digits, and spaces
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    if not text:
        return set()
    return set(text.split())

def jaccard_similarity(set_a, set_b):
    if not set_a and not set_b:
        return 0.0
    inter = len(set_a & set_b)
    union = len(set_a | set_b)
    if union == 0:
        return 0.0
    return inter / union

def compute_within_file_max_jaccard(df, text_cols, id_col_name=None):
    """
    For each row:
      - Concatenate text_cols into a single string
      - Tokenize
      - Compute max Jaccard vs all other rows in the same file
    """
    df = df.copy()

    # Combine selected text columns
    df["combined_text"] = (
        df[text_cols]
        .fillna("")
        .astype(str)
        .agg(" ".join, axis=1)
        .str.strip()
    )

    # Tokenize all rows once
    token_sets = [normalize_and_tokenize(t) for t in df["combined_text"]]

    n = len(df)
    max_jaccard = np.zeros(n, dtype=float)

    for i in tqdm(range(n), desc="Computing max Jaccard within file"):
        s_i = token_sets[i]
        best = 0.0
        for j in range(n):
            if i == j:
                continue
            s_j = token_sets[j]
            sim = jaccard_similarity(s_i, s_j)
            if sim > best:
                best = sim
        max_jaccard[i] = best

    df["max_jaccard_within"] = max_jaccard

    if id_col_name is not None and id_col_name not in df.columns:
        df[id_col_name] = np.arange(1, n + 1)

    return df

# ----------------------------
# Robust Excel reader (handles extra / shifted header rows)
# ----------------------------

def read_excel_with_required_cols(path, required_cols):
    """
    Try to read Excel and ensure required_cols exist as columns.
    Steps:
      1) Try header=0
      2) Try header=1
      3) Fallback: header=None and search for the row that contains all required_cols
    """
    # --- Try header=0 ---
    print(f"\nReading: {path} with header=0")
    df0 = pd.read_excel(path, header=0)
    df0.columns = df0.columns.map(lambda x: str(x).strip())
    print("Columns (header=0):", list(df0.columns))

    if required_cols.issubset(set(df0.columns)):
        print("✅ Found required columns with header=0")
        return df0

    # --- Try header=1 ---
    print("Required columns not found with header=0. Trying header=1...")
    df1 = pd.read_excel(path, header=1)
    df1.columns = df1.columns.map(lambda x: str(x).strip())
    print("Columns (header=1):", list(df1.columns))

    if required_cols.issubset(set(df1.columns)):
        print("✅ Found required columns with header=1")
        return df1

    # --- Fallback: header=None & scan rows for a header row containing required cols ---
    print("Required columns not found with header=0 or header=1.")
    print("Trying header=None and scanning for a row that contains all required columns...")

    df_raw = pd.read_excel(path, header=None)
    n_rows, n_cols = df_raw.shape
    print(f"Raw shape (header=None): {df_raw.shape}")

    header_row_idx = None
    for i in range(n_rows):
        row_vals = [str(x).strip() for x in df_raw.iloc[i].tolist()]
        row_set = set(row_vals)
        if required_cols.issubset(row_set):
            header_row_idx = i
            print(f"✅ Found header row at index {i}: {row_vals}")
            break

    if header_row_idx is None:
        raise KeyError(
            f"Could not find required columns {required_cols} in any row of {path}.\n"
            f"Example of first few rows (header=None):\n{df_raw.head(5)}"
        )

    # Build DataFrame using that row as header
    header_vals = [str(x).strip() for x in df_raw.iloc[header_row_idx].tolist()]
    df2 = df_raw.iloc[header_row_idx + 1 :].copy()
    df2.columns = header_vals
    df2 = df2.reset_index(drop=True)

    print("Columns (from detected header row):", list(df2.columns))

    if not required_cols.issubset(set(df2.columns)):
        raise KeyError(
            f"Even after detecting header row at {header_row_idx}, "
            f"the DataFrame does not contain required columns {required_cols}. "
            f"Final columns: {list(df2.columns)}"
        )

    print("✅ Successfully constructed DataFrame with required columns from detected header row.")
    return df2

# ----------------------------
# GPT: Question + Correct_Answer
# ----------------------------

gpt_path = "Results_GPTa.xlsx"  # your GPT file

required_gpt_cols = {"Question", "Correct_Answer"}

df_gpt = read_excel_with_required_cols(gpt_path, required_gpt_cols)

print("\nFinal GPT columns:", list(df_gpt.columns))

gpt_text_cols = ["Question", "Correct_Answer"]

print("\nComputing Jaccard for GPT (Question + Correct_Answer)...")
df_gpt_jacc = compute_within_file_max_jaccard(
    df_gpt,
    text_cols=gpt_text_cols,
    id_col_name="Item_ID"
)

gpt_out_path = "results_GPT_with_jaccard.xlsx"
df_gpt_jacc.to_excel(gpt_out_path, index=False)
print(f"\nSaved GPT Jaccard results to: {gpt_out_path}")

print("\nGPT max Jaccard summary:")
print(df_gpt_jacc["max_jaccard_within"].describe())

# ----------------------------
# Gemma: Question + Choice_1 + Choice_2 + Choice_3
# ----------------------------

gemma_path = "Gemma_modified.xlsx"  # adjust if needed

required_gemma_cols = {"Question", "Choice_1", "Choice_2", "Choice_3"}

df_gemma = read_excel_with_required_cols(gemma_path, required_gemma_cols)

print("\nFinal Gemma columns:", list(df_gemma.columns))

gemma_text_cols = ["Question", "Choice_1", "Choice_2", "Choice_3"]

print("\nComputing Jaccard for Gemma (Question + 3 choices)...")
df_gemma_jacc = compute_within_file_max_jaccard(
    df_gemma,
    text_cols=gemma_text_cols,
    id_col_name="Item_ID"
)

gemma_out_path = "Gemma_Results_with_jaccard.xlsx"
df_gemma_jacc.to_excel(gemma_out_path, index=False)
print(f"\nSaved Gemma Jaccard results to: {gemma_out_path}")

print("\nGemma max Jaccard summary:")
print(df_gemma_jacc["max_jaccard_within"].describe())



Reading: Results_GPTa.xlsx with header=0
Columns (header=0): ['Row', 'Prompting_strategy', 'Question_Type', 'Question', 'Correct_Answer', 'Word_Difficulty', 'Task_Difficulty', 'Text', 'Overall_Score', 'Grammar_Score', 'Complexity_Score', 'Readability_Score', 'Fluency_Score', 'Error_Count', 'Errors']
✅ Found required columns with header=0

Final GPT columns: ['Row', 'Prompting_strategy', 'Question_Type', 'Question', 'Correct_Answer', 'Word_Difficulty', 'Task_Difficulty', 'Text', 'Overall_Score', 'Grammar_Score', 'Complexity_Score', 'Readability_Score', 'Fluency_Score', 'Error_Count', 'Errors']

Computing Jaccard for GPT (Question + Correct_Answer)...


Computing max Jaccard within file: 100%|███| 3509/3509 [00:09<00:00, 378.80it/s]



Saved GPT Jaccard results to: results_GPT_with_jaccard.xlsx

GPT max Jaccard summary:
count    3509.000000
mean        0.838467
std         0.178802
min         0.256410
25%         0.714286
50%         0.882353
75%         1.000000
max         1.000000
Name: max_jaccard_within, dtype: float64

Reading: Gemma_modified.xlsx with header=0
Columns (header=0): ['Prompting_strategy', 'Question_Type', 'Question', 'Correct_Answer', 'Choice_1', 'Choice_2', 'Choice_3', 'Word_Difficulty', 'Task_Difficulty', 'Text', 'Overall_Score', 'Grammar_Score', 'Complexity_Score', 'Readability_Score', 'Fluency_Score', 'Error_Count']
✅ Found required columns with header=0

Final Gemma columns: ['Prompting_strategy', 'Question_Type', 'Question', 'Correct_Answer', 'Choice_1', 'Choice_2', 'Choice_3', 'Word_Difficulty', 'Task_Difficulty', 'Text', 'Overall_Score', 'Grammar_Score', 'Complexity_Score', 'Readability_Score', 'Fluency_Score', 'Error_Count']

Computing Jaccard for Gemma (Question + 3 choices)...


Computing max Jaccard within file: 100%|███| 2143/2143 [00:03<00:00, 638.71it/s]



Saved Gemma Jaccard results to: Gemma_Results_with_jaccard.xlsx

Gemma max Jaccard summary:
count    2143.000000
mean        0.969555
std         0.123335
min         0.208333
25%         1.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: max_jaccard_within, dtype: float64


In [78]:
import pandas as pd

# -------------------------------------------------------------------
# Load the Jaccard-annotated files (or the original ones if you prefer)
# -------------------------------------------------------------------
gpt_path   = "results_GPT_with_jaccard.xlsx"      # or your original GPT file
gemma_path = "Gemma_Results_with_jaccard.xlsx"    # or Gemma_modified.xlsx

df_gpt   = pd.read_excel(gpt_path)
df_gemma = pd.read_excel(gemma_path)

# -------------------------------------------------------------------
# Build "full item" strings:
#   - GPT:   Question + Correct_Answer
#   - Gemma: Question + Choice_1 + Choice_2 + Choice_3
# -------------------------------------------------------------------
def build_gpt_full_item(row):
    q  = str(row.get("Question", "") or "").strip()
    ca = str(row.get("Correct_Answer", "") or "").strip()
    return (q + " [ANS] " + ca).strip()

def build_gemma_full_item(row):
    q  = str(row.get("Question", "") or "").strip()
    c1 = str(row.get("Choice_1", "") or "").strip()
    c2 = str(row.get("Choice_2", "") or "").strip()
    c3 = str(row.get("Choice_3", "") or "").strip()
    return (q + " [A] " + c1 + " [B] " + c2 + " [C] " + c3).strip()

df_gpt["full_item"]   = df_gpt.apply(build_gpt_full_item, axis=1)
df_gemma["full_item"] = df_gemma.apply(build_gemma_full_item, axis=1)

# -------------------------------------------------------------------
# Exact-duplicate analysis
# -------------------------------------------------------------------
def summarize_exact_duplicates(df, label):
    n_total   = len(df)
    n_unique  = df["full_item"].nunique()
    n_dupes   = n_total - n_unique
    pct_unique = n_unique / n_total * 100
    pct_dupes  = n_dupes / n_total * 100

    print(f"\n=== {label}: Exact full-item overlap ===")
    print(f"Total items:     {n_total}")
    print(f"Unique items:    {n_unique} ({pct_unique:.2f}%)")
    print(f"Exact duplicates:{n_dupes} ({pct_dupes:.2f}%)")

    # Mark which rows are duplicates (beyond the first occurrence)
    df["is_exact_duplicate"] = df.duplicated("full_item", keep="first")
    return df

df_gpt   = summarize_exact_duplicates(df_gpt,   "GPT")
df_gemma = summarize_exact_duplicates(df_gemma, "Gemma")

# Save back if you want the flags
df_gpt.to_excel("results_GPT_with_jaccard_and_exact.xlsx", index=False)
df_gemma.to_excel("Gemma_Results_with_jaccard_and_exact.xlsx", index=False)
print("\nSaved updated files with 'full_item' and 'is_exact_duplicate' columns.")



=== GPT: Exact full-item overlap ===
Total items:     3509
Unique items:    2613 (74.47%)
Exact duplicates:896 (25.53%)

=== Gemma: Exact full-item overlap ===
Total items:     2143
Unique items:    393 (18.34%)
Exact duplicates:1750 (81.66%)

Saved updated files with 'full_item' and 'is_exact_duplicate' columns.
